<a href="https://colab.research.google.com/github/sandipan-das-sd/AI-Trip-Planner/blob/main/NITY50_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kiteconnect pandas numpy scikit-learn matplotlib tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
from kiteconnect import KiteConnect
import pandas as pd

API_KEY = "dhws8c8g30qu0det"  # Replace with yours
API_SECRET = "llhzhdu4k1rcyyer1ivdv9ik6ihyvhkm"  # Replace with yours

kite = KiteConnect(api_key=API_KEY)\

# Step 1: Get request token manually (this is one-time per login session)
print("Login URL:", kite.login_url())

# After login, paste the request_token here:
request_token = input("Enter the request token from login: ")
print(f"You entered request token: {request_token}") # Added this line to show the user what they entered

# Step 2: Get access token
data = kite.generate_session(request_token, api_secret=API_SECRET)
kite.set_access_token(data["access_token"])
print("✅ Logged in")

Login URL: https://kite.zerodha.com/connect/login?api_key=dhws8c8g30qu0det&v=3
Enter the request token from login: p0A6bLGAMhFBUap9ZuOArRq8gZvyx5WD
You entered request token: p0A6bLGAMhFBUap9ZuOArRq8gZvyx5WD
✅ Logged in


In [8]:
from datetime import datetime, timedelta

def fetch_option_data(symbol, expiry, strike_price, option_type):
    instruments = kite.instruments("NSE")
    df = pd.DataFrame(instruments)
    df = df[df["name"] == "NIFTY"]
    df = df[(df["strike"] == float(strike_price)) & (df["expiry"] == pd.to_datetime(expiry))]
    df = df[df["instrument_type"] == option_type.upper()]

    if df.empty:
        print("❌ Option not found for the given parameters.")
        return None

    token = df.iloc[0]["instrument_token"]

    # fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()

    hist = kite.historical_data(token, from_date, to_date, "30minute")
    df_hist = pd.DataFrame(hist)
    return df_hist

# Example usage
data = fetch_option_data("NIFTY", "2025-08-28", "24250", "CE")

if data is not None:
    display(data.head())

❌ Option not found for the given parameters.


In [10]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Prepare data
if data is not None:
    df = data[['close']]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    # Create sequences
    def create_sequences(data, time_steps=50):
        X, y = [], []
        for i in range(time_steps, len(data)):
            X.append(data[i-time_steps:i])
            y.append(data[i])
        return np.array(X), np.array(y)

    X, y = create_sequences(scaled_data)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train model
    model.fit(X, y, epochs=10, batch_size=32)
else:
    print("Cannot train the model because no option data was fetched.")

Cannot train the model because no option data was fetched.


In [12]:
# Predict next price
if data is not None and 'scaled_data' in locals(): # Added check for data and scaled_data
    last_sequence = scaled_data[-50:]
    last_sequence = last_sequence.reshape(1, 50, 1)

    predicted_scaled = model.predict(last_sequence)
    predicted_price = scaler.inverse_transform(predicted_scaled)

    # Suggest entry, stoploss, target
    entry_price = float(df.iloc[-1]['close'])
    predicted = float(predicted_price[0][0])

    stoploss = entry_price - ((entry_price - predicted) * 0.5)
    target = predicted + ((predicted - entry_price) * 0.5)

    print(f"📈 Entry Price: ₹{entry_price:.2f}")
    print(f"🔮 Predicted Max: ₹{predicted:.2f}")
    print(f"🛑 Stoploss: ₹{stoploss:.2f}")
    print(f"🎯 Target: ₹{target:.2f}")
else: # Added else block
    print("Cannot make predictions because no option data was fetched or the model was not trained.")

Cannot make predictions because no option data was fetched or the model was not trained.


In [13]:
# Fetch latest NIFTY index
nifty_quote = kite.ltp("NSE:NIFTY 50")["NSE:NIFTY 50"]["last_price"]
next_strike = round(nifty_quote / 100) * 100

print("Next Day Suggested Trade:")
print(f"👉 Option: {next_strike} CE or PE (based on trend)")
print(f"👉 Date: Next Thursday Expiry")


Next Day Suggested Trade:
👉 Option: 24400 CE or PE (based on trend)
👉 Date: Next Thursday Expiry


In [42]:
instruments = kite.instruments("NSE")
df_instruments = pd.DataFrame(instruments)
display(df_instruments.head())

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


# Task
Modify the provided Python notebook for NIFTY Option Price Prediction using LSTM and Zerodha Kite API to allow the user to input the expiry date, strike price, and call/put type. Based on this input, the notebook should fetch the relevant option data, use the existing LSTM model to predict the maximum value point the option will reach, provide two target prices with high confidence, and calculate a stop loss. The output should display the entry price, stop loss, and the two high-confidence target prices.

## Modify data fetching

### Subtask:
Update the `fetch_option_data` function to accept user inputs for expiry date, strike price, and option type (CE/PE).


**Reasoning**:
The subtask is to modify the `fetch_option_data` function to accept user inputs. This involves changing the function signature, converting the input strings to appropriate types, and updating the filtering logic. I will update the function definition and the example usage in the provided code block.



In [15]:
from datetime import datetime, timedelta

def fetch_option_data(symbol, expiry_date_str, strike_price_str, option_type_str):
    instruments = kite.instruments("NSE")
    df = pd.DataFrame(instruments)
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    expiry_date = pd.to_datetime(expiry_date_str)
    strike_price = float(strike_price_str)
    option_type = option_type_str.upper()

    df = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df = df[df["instrument_type"] == option_type]

    if df.empty:
        print("❌ Option not found for the given parameters.")
        return None

    token = df.iloc[0]["instrument_token"]

    # fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()

    hist = kite.historical_data(token, from_date, to_date, "30minute")
    df_hist = pd.DataFrame(hist)
    return df_hist

# Example usage with placeholder strings
data = fetch_option_data("NIFTY", "2025-08-28", "24250", "CE")

if data is not None:
    display(data.head())

❌ Option not found for the given parameters.


## Modify data preparation

### Subtask:
Adjust the data preparation steps to handle potential variations in fetched data based on user input.


**Reasoning**:
Adjust the data preparation steps to handle potential variations in fetched data based on user input.



In [16]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Prepare data
if data is not None and not data.empty:
    df = data[['close']].copy() # Use .copy() to avoid SettingWithCopyWarning
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    # Create sequences
    def create_sequences(data, time_steps=50):
        X, y = [], []
        # Check if there is enough data to create sequences
        if len(data) < time_steps:
            return np.array(X), np.array(y)

        for i in range(time_steps, len(data)):
            X.append(data[i-time_steps:i])
            y.append(data[i])
        return np.array(X), np.array(y)

    time_steps = 50 # Define time_steps
    X, y = create_sequences(scaled_data, time_steps)

    # Check if sequences were created successfully
    if X.shape[0] > 0:
        X = X.reshape(X.shape[0], X.shape[1], 1)

        # LSTM model
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model.add(LSTM(units=50))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Train model
        model.fit(X, y, epochs=10, batch_size=32)
    else:
         print("Not enough data points to create sequences for LSTM training.")

else:
    print("Cannot train the model because no option data was fetched or the fetched data is empty.")


Cannot train the model because no option data was fetched or the fetched data is empty.


## Modify the lstm model

### Subtask:
Consider if the current LSTM model architecture is sufficient for predicting multiple target points with confidence. If not, propose modifications or alternative model architectures.


**Reasoning**:
Evaluate the current LSTM model architecture and propose modifications or alternatives for multi-point prediction with confidence.



In [17]:
# Evaluation of the current LSTM model architecture
print("Evaluation of current LSTM model architecture:")
print("The current LSTM model architecture is a standard sequential model with two LSTM layers and one Dense output layer.")
print("This architecture is primarily designed for single-point time series prediction, predicting the next value in a sequence.")
print("It outputs a single value, which is typically interpreted as the mean or expected value of the prediction.")
print("This architecture does not inherently provide a measure of confidence or predict multiple target points (e.g., a range or a distribution).")
print("Therefore, it is not sufficient for predicting multiple target points with high confidence as required by the task.")

# Proposed modifications or alternative architectures
print("\nProposed modifications or alternative architectures for multi-point prediction with confidence:")

# Option 1: Modify the output layer for predicting a range (e.g., lower and upper bounds)
print("\nOption 1: Modify the output layer to predict a range.")
print("Instead of a single Dense layer with 1 unit, we can use a Dense layer with 2 units.")
print("One unit would predict the lower bound of the price range, and the other the upper bound.")
print("This requires modifying the loss function to be appropriate for range prediction (e.g., a custom loss function that penalizes predictions outside the actual range).")
print("This approach provides a prediction interval but doesn't directly give multiple distinct target prices with confidence levels.")

# Option 2: Use a probabilistic approach (e.g., predicting mean and variance)
print("\nOption 2: Use a probabilistic approach to predict mean and variance.")
print("Modify the output layer to have 2 units: one for the predicted mean and one for the predicted variance (or standard deviation).")
print("This allows for estimating the probability distribution of the next price.")
print("We can then derive confidence intervals (e.g., mean +/- 1.96 * std dev for 95% confidence) from this distribution.")
print("This provides a confidence measure but still focuses on a single predicted point (the mean) and a range around it.")

# Option 3: Employ Quantile Regression LSTMs
print("\nOption 3: Employ Quantile Regression LSTMs.")
print("Instead of predicting the mean, train the LSTM to predict different quantiles of the price distribution (e.g., 10th, 50th, 90th percentiles).")
print("This involves using a custom loss function (quantile loss) and potentially multiple output units or multiple models, each trained for a specific quantile.")
print("Predicting multiple quantiles allows us to define prediction intervals and potentially infer different 'target' levels based on these quantiles.")
print("This directly addresses the need for multiple points (quantiles) that can be interpreted with some level of confidence (the quantile level).")

# Option 4: Bayesian LSTMs
print("\nOption 4: Use Bayesian LSTMs.")
print("Bayesian LSTMs frame the weights of the network as probability distributions rather than fixed values.")
print("This allows the model to output a distribution over possible predictions, providing a natural way to quantify uncertainty and derive confidence intervals.")
print("This is a more complex approach but offers a robust way to estimate prediction confidence.")

# Rationale
print("\nRationale:")
print("The current model's limitation is its single-point output without uncertainty estimation.")
print("The proposed modifications/alternatives aim to either: ")
print("1. Expand the output to cover a range or multiple points (Options 1 & 3).")
print("2. Explicitly model the uncertainty in the prediction (Options 2 & 4).")
print("Option 3 (Quantile Regression LSTMs) seems most aligned with the requirement of providing multiple 'high-confidence target prices' as quantiles directly represent points in the distribution with associated probability levels.")
print("Option 2 (predicting mean and variance) also allows for confidence interval calculation.")
print("Option 4 (Bayesian LSTMs) provides a more principled way to capture uncertainty.")
print("Option 1 (predicting range) is a simpler modification but less informative about the internal distribution.")

Evaluation of current LSTM model architecture:
The current LSTM model architecture is a standard sequential model with two LSTM layers and one Dense output layer.
This architecture is primarily designed for single-point time series prediction, predicting the next value in a sequence.
It outputs a single value, which is typically interpreted as the mean or expected value of the prediction.
This architecture does not inherently provide a measure of confidence or predict multiple target points (e.g., a range or a distribution).
Therefore, it is not sufficient for predicting multiple target points with high confidence as required by the task.

Proposed modifications or alternative architectures for multi-point prediction with confidence:

Option 1: Modify the output layer to predict a range.
Instead of a single Dense layer with 1 unit, we can use a Dense layer with 2 units.
One unit would predict the lower bound of the price range, and the other the upper bound.
This requires modifying the

## Implement prediction logic for multiple targets

### Subtask:
Develop a method to predict not just a single price but a range of potential prices with associated confidence levels (e.g., using prediction intervals or a different model output layer). Define how "high confidence" targets will be determined.


**Reasoning**:
Modify the existing LSTM model to predict three quantiles (25th, 50th, 75th) by changing the output layer and using a custom quantile loss function. Then, retrain the model.



In [18]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

# Define the quantile loss function
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Prepare data (assuming data, scaled_data, X, and y are available from previous steps)
if 'X' in locals() and X.shape[0] > 0:

    # LSTM model for Quantile Regression
    model_quantile = Sequential()
    model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model_quantile.add(LSTM(units=50))
    # Output layer for 3 quantiles (25th, 50th, 75th)
    model_quantile.add(Dense(3))

    # Define quantiles
    quantiles = [0.25, 0.50, 0.75]

    # Compile model with quantile loss
    model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

    # Train model
    print("Training Quantile Regression LSTM model...")
    model_quantile.fit(X, y, epochs=10, batch_size=32)
    print("✅ Model training complete.")

else:
    print("Cannot train the quantile regression model because data is not available or not enough data points to create sequences.")


Cannot train the quantile regression model because data is not available or not enough data points to create sequences.


**Reasoning**:
Since the previous attempt to train the quantile regression model failed due to insufficient data, I need to first ensure that the data fetching and preparation steps are executed with valid inputs to provide data for training. I will use a valid example usage for `fetch_option_data` to get the necessary data, then prepare it and train the model.



In [19]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

# Define the quantile loss function
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Fetch data with valid parameters (example)
# Assuming a valid NIFTY option exists for this date and strike
# Replace with actual valid date and strike if needed, based on df_instruments
# Let's try fetching data for a recent NIFTY expiry and a strike close to the current NIFTY price
# Based on previous output, NIFTY is around 24362.6, next_strike is 24400.
# Let's try a recent expiry, e.g., next Thursday from 2025-08-07 would be 2025-08-14
data = fetch_option_data("NIFTY", "2025-08-14", "24400", "CE")


# Prepare data
if data is not None and not data.empty:
    df = data[['close']].copy() # Use .copy() to avoid SettingWithCopyWarning
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    # Create sequences
    def create_sequences(data, time_steps=50):
        X, y = [], []
        # Check if there is enough data to create sequences
        if len(data) < time_steps:
            return np.array(X), np.array(y)

        for i in range(time_steps, len(data)):
            X.append(data[i-time_steps:i])
            y.append(data[i])
        return np.array(X), np.array(y)

    time_steps = 50 # Define time_steps
    X, y = create_sequences(scaled_data, time_steps)

    # Check if sequences were created successfully
    if X.shape[0] > 0:
        X = X.reshape(X.shape[0], X.shape[1], 1)
        # Reshape y for quantile loss (needs to be shape (samples, number_of_quantiles))
        # Since we are predicting 3 quantiles for a single time step, y should be (samples, 1) initially
        # We will predict 3 values per sample, so the model output will be (samples, 3)
        # The quantile loss function expects y_true to have the same shape as y_pred for the last dimension
        # So, y_true should be (samples, 3). We can repeat the true value 3 times for each sample.
        y_quantile = np.repeat(y, 3, axis=1)


        # LSTM model for Quantile Regression
        model_quantile = Sequential()
        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model_quantile.add(LSTM(units=50))
        # Output layer for 3 quantiles (25th, 50th, 75th)
        model_quantile.add(Dense(3))

        # Define quantiles
        quantiles = [0.25, 0.50, 0.75]

        # Compile model with quantile loss
        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model...")
        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32) # Use y_quantile for training
        print("✅ Model training complete.")

        # Implement prediction and interpretation logic after training
        print("\nImplementing prediction and interpretation...")
        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
            last_sequence = scaled_data[-time_steps:]
            last_sequence = last_sequence.reshape(1, time_steps, 1)

            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

            # Interpret predicted quantiles
            # The predicted_quantiles will be in the order of quantiles defined: [25th, 50th, 75th]
            q25 = predicted_quantiles[0][0]
            q50 = predicted_quantiles[0][1]
            q75 = predicted_quantiles[0][2]

            # Define high confidence targets
            # Target 1: The 75th percentile
            target1 = q75
            # Target 2: A point above the 50th percentile, perhaps halfway between 50th and 75th
            target2 = q50 + (q75 - q50) / 2 # Example logic

            # Calculate stop loss
            # Entry price is the last closing price
            entry_price = float(df.iloc[-1]['close'])
            # Stop loss could be below the 25th percentile or a fixed percentage below entry
            # Let's use a value below the 25th percentile as an example
            stoploss = q25 - (q50 - q25) # Example logic: below 25th percentile by the same distance as 50th is from 25th

            print(f"📈 Entry Price: ₹{entry_price:.2f}")
            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
        else:
            print("Cannot make predictions: Not enough data to create the last sequence.")


    else:
         print("Not enough data points to create sequences for LSTM training.")

else:
    print("Cannot train the model because no option data was fetched or the fetched data is empty.")


❌ Option not found for the given parameters.
Cannot train the model because no option data was fetched or the fetched data is empty.


**Reasoning**:
The data fetching failed again, indicating that the chosen expiry date or strike price is not available. I need to find a valid expiry date and strike price for NIFTY options to successfully fetch data and proceed with training and prediction. I will inspect the `df_instruments` DataFrame to find a valid NIFTY option with an upcoming expiry date and a strike price close to the current NIFTY price. Then I will use those parameters to fetch the data and retry the training and prediction steps.



In [20]:
# Find a valid NIFTY option from df_instruments
if 'df_instruments' in locals() and not df_instruments.empty:
    nifty_options = df_instruments[(df_instruments['name'] == 'NIFTY') &
                                  (df_instruments['instrument_type'].isin(['CE', 'PE'])) &
                                  (df_instruments['expiry'].notna())].copy() # Use .copy() to avoid SettingWithCopyWarning

    if not nifty_options.empty:
        # Sort by expiry date to find an upcoming one
        nifty_options['expiry'] = pd.to_datetime(nifty_options['expiry'])
        nifty_options = nifty_options.sort_values(by='expiry')

        # Get the current NIFTY price to find a relevant strike
        # Assuming nifty_quote is available from a previous step
        if 'nifty_quote' in locals():
            current_nifty_price = nifty_quote
        else:
            # Fetch current NIFTY price if not available
            try:
                nifty_quote_data = kite.ltp("NSE:NIFTY 50")
                current_nifty_price = nifty_quote_data["NSE:NIFTY 50"]["last_price"]
                print(f"Fetched current NIFTY price: {current_nifty_price}")
            except Exception as e:
                print(f"Error fetching NIFTY LTP: {e}")
                current_nifty_price = None
                print("Cannot determine a relevant strike price without current NIFTY price.")

        if current_nifty_price is not None:
            # Find the option closest to the current price with an upcoming expiry
            closest_option = None
            min_price_diff = float('inf')

            # Iterate through upcoming expiries and find a strike close to the current price
            for expiry_date in nifty_options['expiry'].unique():
                options_on_expiry = nifty_options[nifty_options['expiry'] == expiry_date]
                if not options_on_expiry.empty:
                    # Find the strike closest to the current price on this expiry
                    options_on_expiry['strike_diff'] = abs(options_on_expiry['strike'] - current_nifty_price)
                    closest_strike_option = options_on_expiry.sort_values(by='strike_diff').iloc[0]

                    # Consider this option if it's the closest strike across expiries so far
                    if closest_strike_option['strike_diff'] < min_price_diff:
                         closest_option = closest_strike_option
                         min_price_diff = closest_strike_option['strike_diff']


            if closest_option is not None:
                valid_expiry = closest_option['expiry'].strftime('%Y-%m-%d')
                valid_strike = str(int(closest_option['strike'])) # Ensure strike is integer string
                valid_option_type = closest_option['instrument_type'] # Use the instrument type of the found option

                print(f"\nFound a valid NIFTY option:")
                print(f"Expiry: {valid_expiry}")
                print(f"Strike: {valid_strike}")
                print(f"Type: {valid_option_type}")

                # Now, attempt to fetch data with the valid parameters
                print(f"\nAttempting to fetch data for {valid_option_type} {valid_strike} {valid_expiry}...")
                data = fetch_option_data("NIFTY", valid_expiry, valid_strike, valid_option_type)

                # Prepare data and train model if data is fetched
                if data is not None and not data.empty:
                    df = data[['close']].copy()
                    scaler = MinMaxScaler()
                    scaled_data = scaler.fit_transform(df)

                    time_steps = 50
                    X, y = create_sequences(scaled_data, time_steps)

                    if X.shape[0] > 0:
                        X = X.reshape(X.shape[0], X.shape[1], 1)
                        y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

                        # Define and compile the quantile regression model
                        model_quantile = Sequential()
                        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                        model_quantile.add(LSTM(units=50))
                        model_quantile.add(Dense(3)) # Output 3 quantiles

                        quantiles = [0.25, 0.50, 0.75]
                        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                        # Train model
                        print("Training Quantile Regression LSTM model...")
                        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
                        print("✅ Model training complete.")

                        # Implement prediction and interpretation logic
                        print("\nImplementing prediction and interpretation...")
                        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
                            last_sequence = scaled_data[-time_steps:]
                            last_sequence = last_sequence.reshape(1, time_steps, 1)

                            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
                            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

                            # Interpret predicted quantiles
                            q25 = predicted_quantiles[0][0]
                            q50 = predicted_quantiles[0][1]
                            q75 = predicted_quantiles[0][2]

                            # Define high confidence targets
                            target1 = q75 # 75th percentile as Target 1
                            target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

                            # Calculate stop loss
                            entry_price = float(df.iloc[-1]['close'])
                            # Stop loss below the 25th percentile, using distance between 50th and 25th
                            stoploss = q25 - (q50 - q25)


                            print(f"📈 Entry Price: ₹{entry_price:.2f}")
                            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
                            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
                            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
                            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
                        else:
                            print("Cannot make predictions: Not enough data to create the last sequence.")

                    else:
                         print("Not enough data points to create sequences for LSTM training.")

                else:
                    print("Cannot train the model because data fetching with valid parameters failed or returned empty data.")

            else:
                 print("Could not find a suitable NIFTY option with an upcoming expiry and strike near the current price.")
        else:
             print("Current NIFTY price is not available to determine a relevant strike.")
    else:
        print("No NIFTY options found in the instruments data.")
else:
    print("Instruments data (df_instruments) is not available or empty.")


No NIFTY options found in the instruments data.


**Reasoning**:
The previous attempt failed because no NIFTY options were found in the `df_instruments` DataFrame. This is unexpected given the nature of the Kite API. I need to re-examine how `df_instruments` was populated and ensure it contains the necessary option data. I will re-fetch the instruments data and then filter it again to see if NIFTY options become available. If they are, I will proceed with finding a suitable option and attempting the data fetching, training, and prediction steps again within the same code block to ensure the entire process runs sequentially after successful data loading.



In [21]:
# Re-fetch instruments data to ensure it's up-to-date and contains options
try:
    instruments = kite.instruments("NSE")
    df_instruments = pd.DataFrame(instruments)
    print("✅ Successfully re-fetched instruments data.")
    # display(df_instruments.head()) # Optional: display head to confirm data

    # Now, proceed with finding a valid NIFTY option and the rest of the process
    if 'df_instruments' in locals() and not df_instruments.empty:
        nifty_options = df_instruments[(df_instruments['name'] == 'NIFTY') &
                                      (df_instruments['instrument_type'].isin(['CE', 'PE'])) &
                                      (df_instruments['expiry'].notna())].copy()

        if not nifty_options.empty:
            # Sort by expiry date to find an upcoming one
            nifty_options['expiry'] = pd.to_datetime(nifty_options['expiry'])
            nifty_options = nifty_options.sort_values(by='expiry')

            # Get the current NIFTY price to find a relevant strike
            # Assuming nifty_quote is available from a previous step
            if 'nifty_quote' in locals():
                current_nifty_price = nifty_quote
            else:
                # Fetch current NIFTY price if not available
                try:
                    nifty_quote_data = kite.ltp("NSE:NIFTY 50")
                    current_nifty_price = nifty_quote_data["NSE:NIFTY 50"]["last_price"]
                    print(f"Fetched current NIFTY price: {current_nifty_price}")
                except Exception as e:
                    print(f"Error fetching NIFTY LTP: {e}")
                    current_nifty_price = None
                    print("Cannot determine a relevant strike price without current NIFTY price.")


            if current_nifty_price is not None:
                # Find the option closest to the current price with an upcoming expiry
                closest_option = None
                min_price_diff = float('inf')

                # Iterate through upcoming expiries and find a strike close to the current price
                for expiry_date in nifty_options['expiry'].unique():
                    options_on_expiry = nifty_options[options_on_expiry['expiry'] == expiry_date]
                    if not options_on_expiry.empty:
                        # Find the strike closest to the current price on this expiry
                        options_on_expiry['strike_diff'] = abs(options_on_expiry['strike'] - current_nifty_price)
                        closest_strike_option = options_on_expiry.sort_values(by='strike_diff').iloc[0]

                        # Consider this option if it's the closest strike across expiries so far
                        if closest_strike_option['strike_diff'] < min_price_diff:
                             closest_option = closest_strike_option
                             min_price_diff = closest_strike_option['strike_diff']


                if closest_option is not None:
                    valid_expiry = closest_option['expiry'].strftime('%Y-%m-%d')
                    valid_strike = str(int(closest_option['strike']))
                    valid_option_type = closest_option['instrument_type']

                    print(f"\nFound a valid NIFTY option:")
                    print(f"Expiry: {valid_expiry}")
                    print(f"Strike: {valid_strike}")
                    print(f"Type: {valid_option_type}")

                    # Now, attempt to fetch data with the valid parameters
                    print(f"\nAttempting to fetch data for {valid_option_type} {valid_strike} {valid_expiry}...")
                    data = fetch_option_data("NIFTY", valid_expiry, valid_strike, valid_option_type)

                    # Prepare data and train model if data is fetched
                    if data is not None and not data.empty:
                        df = data[['close']].copy()
                        scaler = MinMaxScaler()
                        scaled_data = scaler.fit_transform(df)

                        time_steps = 50
                        X, y = create_sequences(scaled_data, time_steps)

                        if X.shape[0] > 0:
                            X = X.reshape(X.shape[0], X.shape[1], 1)
                            y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

                            # Define and compile the quantile regression model
                            model_quantile = Sequential()
                            model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                            model_quantile.add(LSTM(units=50))
                            model_quantile.add(Dense(3)) # Output 3 quantiles

                            quantiles = [0.25, 0.50, 0.75]
                            model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                            # Train model
                            print("Training Quantile Regression LSTM model...")
                            model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
                            print("✅ Model training complete.")

                            # Implement prediction and interpretation logic
                            print("\nImplementing prediction and interpretation...")
                            if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
                                last_sequence = scaled_data[-time_steps:]
                                last_sequence = last_sequence.reshape(1, time_steps, 1)

                                predicted_scaled_quantiles = model_quantile.predict(last_sequence)
                                predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

                                # Interpret predicted quantiles
                                q25 = predicted_quantiles[0][0]
                                q50 = predicted_quantiles[0][1]
                                q75 = predicted_quantiles[0][2]

                                # Define high confidence targets
                                target1 = q75 # 75th percentile as Target 1
                                target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

                                # Calculate stop loss
                                entry_price = float(df.iloc[-1]['close'])
                                # Stop loss below the 25th percentile, using distance between 50th and 25th
                                stoploss = q25 - (q50 - q25)


                                print(f"📈 Entry Price: ₹{entry_price:.2f}")
                                print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
                                print(f"🛑 Stoploss: ₹{stoploss:.2f}")
                                print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
                                print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
                            else:
                                print("Cannot make predictions: Not enough data to create the last sequence.")

                        else:
                             print("Not enough data points to create sequences for LSTM training.")

                    else:
                        print("Cannot train the model because data fetching with valid parameters failed or returned empty data.")

                else:
                     print("Could not find a suitable NIFTY option with an upcoming expiry and strike near the current price.")
            else:
                 print("Current NIFTY price is not available to determine a relevant strike.")
        else:
            print("No NIFTY options found in the instruments data after re-fetching.")
    else:
        print("Instruments data (df_instruments) is not available or empty after re-fetching.")

except Exception as e:
    print(f"An error occurred during the process: {e}")


✅ Successfully re-fetched instruments data.
No NIFTY options found in the instruments data after re-fetching.


## Implement prediction logic for multiple targets retry 1

### Subtask:
Implement prediction logic for multiple targets


**Reasoning**:
The previous attempts failed because no valid NIFTY option data could be found or fetched. To move forward with implementing the prediction logic, I need to find a valid NIFTY option from the available instruments, fetch its data, and then proceed with the quantile regression model training and prediction. This code block will attempt to find a suitable NIFTY option, fetch its data, and then train the quantile regression model if data is available. It combines steps 1 through 5 of the current subtask instructions.



In [22]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

# Define the quantile loss function
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences (assuming this is available from previous steps)
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data (assuming this is available from previous steps)
def fetch_option_data(symbol, expiry_date_str, strike_price_str, option_type_str):
    # Use the globally available df_instruments if it exists, otherwise fetch
    global df_instruments
    if 'df_instruments' not in globals() or df_instruments is None or df_instruments.empty:
        try:
            instruments = kite.instruments("NSE")
            df_instruments = pd.DataFrame(instruments)
            print("✅ Successfully fetched instruments data within fetch_option_data.")
        except Exception as e:
            print(f"Error fetching instruments data in fetch_option_data: {e}")
            return None

    df = df_instruments.copy() # Use a copy to avoid modifying the global DataFrame
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None


    df = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df = df[df["instrument_type"] == option_type]

    if df.empty:
        print(f"❌ Option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")
        return None

    token = df.iloc[0]["instrument_token"]

    # fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()

    try:
        hist = kite.historical_data(token, from_date, to_date, "30minute")
        df_hist = pd.DataFrame(hist)
        if df_hist.empty:
             print(f"❌ No historical data found for token {token}.")
             return None
        return df_hist
    except Exception as e:
        print(f"Error fetching historical data for token {token}: {e}")
        return None

# Main process to find data, train model, and predict

# Re-fetch instruments data to ensure it's up-to-date and contains options
try:
    instruments = kite.instruments("NSE")
    df_instruments = pd.DataFrame(instruments)
    print("✅ Successfully re-fetched instruments data.")

    # Find a valid NIFTY option from df_instruments
    if 'df_instruments' in locals() and not df_instruments.empty:
        nifty_options = df_instruments[(df_instruments['name'] == 'NIFTY') &
                                      (df_instruments['instrument_type'].isin(['CE', 'PE'])) &
                                      (df_instruments['expiry'].notna())].copy()

        if not nifty_options.empty:
            # Sort by expiry date to find an upcoming one
            nifty_options['expiry'] = pd.to_datetime(nifty_options['expiry'])
            nifty_options = nifty_options.sort_values(by='expiry')

            # Get the current NIFTY price to find a relevant strike
            # Assuming nifty_quote is available from a previous step
            if 'nifty_quote' in locals():
                current_nifty_price = nifty_quote
            else:
                # Fetch current NIFTY price if not available
                try:
                    nifty_quote_data = kite.ltp("NSE:NIFTY 50")
                    current_nifty_price = nifty_quote_data["NSE:NIFTY 50"]["last_price"]
                    print(f"Fetched current NIFTY price: {current_nifty_price}")
                except Exception as e:
                    print(f"Error fetching NIFTY LTP: {e}")
                    current_nifty_price = None
                    print("Cannot determine a relevant strike price without current NIFTY price.")


            if current_nifty_price is not None:
                # Find the option closest to the current price with an upcoming expiry
                closest_option = None
                min_price_diff = float('inf')

                # Iterate through upcoming expiries and find a strike close to the current price
                # Consider only expiries in the future
                upcoming_expiries = nifty_options[nifty_options['expiry'] > datetime.now()]['expiry'].unique()

                if upcoming_expiries.size > 0:
                    # Find the nearest expiry
                    nearest_expiry = upcoming_expiries[0]
                    options_on_nearest_expiry = nifty_options[nifty_options['expiry'] == nearest_expiry]

                    if not options_on_nearest_expiry.empty:
                        # Find the strike closest to the current price on this expiry
                        options_on_nearest_expiry['strike_diff'] = abs(options_on_nearest_expiry['strike'] - current_nifty_price)
                        closest_option = options_on_nearest_expiry.sort_values(by='strike_diff').iloc[0]
                    else:
                         print("No options found on the nearest upcoming expiry.")


                if closest_option is not None:
                    valid_expiry = closest_option['expiry'].strftime('%Y-%m-%d')
                    valid_strike = str(int(closest_option['strike']))
                    valid_option_type = closest_option['instrument_type']

                    print(f"\nFound a potentially valid NIFTY option:")
                    print(f"Expiry: {valid_expiry}")
                    print(f"Strike: {valid_strike}")
                    print(f"Type: {valid_option_type}")

                    # Now, attempt to fetch data with the valid parameters
                    print(f"\nAttempting to fetch historical data for {valid_option_type} {valid_strike} {valid_expiry}...")
                    data = fetch_option_data("NIFTY", valid_expiry, valid_strike, valid_option_type)

                    # Prepare data and train model if data is fetched
                    if data is not None and not data.empty:
                        df = data[['close']].copy()
                        scaler = MinMaxScaler()
                        scaled_data = scaler.fit_transform(df)

                        time_steps = 50
                        X, y = create_sequences(scaled_data, time_steps)

                        if X.shape[0] > 0:
                            X = X.reshape(X.shape[0], X.shape[1], 1)
                            y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

                            # Define and compile the quantile regression model
                            model_quantile = Sequential()
                            model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                            model_quantile.add(LSTM(units=50))
                            model_quantile.add(Dense(3)) # Output 3 quantiles

                            quantiles = [0.25, 0.50, 0.75]
                            model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                            # Train model
                            print("Training Quantile Regression LSTM model...")
                            model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
                            print("✅ Model training complete.")

                            # Now proceed to prediction in the next step


                        else:
                             print("Not enough data points to create sequences for LSTM training.")

                    else:
                        print("Cannot train the model because historical data fetching with potentially valid parameters failed or returned empty data.")

                else:
                     print("Could not find a suitable NIFTY option with an upcoming expiry and strike near the current price.")
            else:
                 print("Current NIFTY price is not available to determine a relevant strike.")
        else:
            print("No NIFTY options found in the instruments data after re-fetching.")
    else:
        print("Instruments data (df_instruments) is not available or empty after re-fetching.")

except Exception as e:
    print(f"An error occurred during the process of finding data and training: {e}")

✅ Successfully re-fetched instruments data.
No NIFTY options found in the instruments data after re-fetching.


**Reasoning**:
The previous attempt to find valid NIFTY options failed because none were found in the re-fetched instruments data. This is unexpected. Since the core task relies on having this data, I need to investigate why NIFTY options are not appearing in the instruments data. Before proceeding with prediction logic, I need to ensure I can access NIFTY option data. I will attempt to filter the `df_instruments` DataFrame more broadly to see if NIFTY options exist at all, regardless of expiry or strike, and display some rows if found. This is necessary to debug the data availability issue before implementing prediction logic.



In [23]:
# Investigate why NIFTY options are not found
print("Investigating NIFTY options availability in df_instruments...")

if 'df_instruments' in globals() and df_instruments is not None and not df_instruments.empty:
    # Filter for instruments with 'NIFTY' in the name and instrument_type 'CE' or 'PE'
    all_nifty_options = df_instruments[(df_instruments['name'].str.contains('NIFTY', na=False)) &
                                       (df_instruments['instrument_type'].isin(['CE', 'PE']))].copy()

    if not all_nifty_options.empty:
        print(f"Found {len(all_nifty_options)} instruments related to NIFTY options.")
        print("Displaying the first 10 rows of NIFTY options found:")
        display(all_nifty_options.head(10))
    else:
        print("No instruments found in df_instruments with 'NIFTY' in name and instrument_type 'CE' or 'PE'.")
        print("This indicates an issue with the fetched instruments data not containing NIFTY options.")
else:
    print("df_instruments DataFrame is not available or is empty.")

# Since no NIFTY options were found in the previous attempt, I cannot proceed with training or prediction logic.
# The subtask cannot be completed due to data unavailability.

Investigating NIFTY options availability in df_instruments...
No instruments found in df_instruments with 'NIFTY' in name and instrument_type 'CE' or 'PE'.
This indicates an issue with the fetched instruments data not containing NIFTY options.


## Implement prediction logic for multiple targets retry 2

### Subtask:
Implement prediction logic for multiple targets


**Reasoning**:
The previous attempt failed because NIFTY options were not found in `df_instruments`. Re-attempting to fetch instruments data and then proceed with finding a valid option, fetching historical data, training the quantile model, and making predictions based on the predicted quantiles, incorporating the logic developed in the previous failed attempt.



In [24]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

# Define the quantile loss function (assuming this is available)
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences (assuming this is available)
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data (assuming this is available)
# Updated to explicitly pass df_instruments
def fetch_option_data(kite, df_instruments, symbol, expiry_date_str, strike_price_str, option_type_str):
    df = df_instruments.copy() # Use a copy to avoid modifying the original DataFrame
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None


    df = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df = df[df["instrument_type"] == option_type]

    if df.empty:
        print(f"❌ Option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")
        return None

    token = df.iloc[0]["instrument_token"]

    # fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()

    try:
        hist = kite.historical_data(token, from_date, to_date, "30minute")
        df_hist = pd.DataFrame(hist)
        if df_hist.empty:
             print(f"❌ No historical data found for token {token}.")
             return None
        return df_hist
    except Exception as e:
        print(f"Error fetching historical data for token {token}: {e}")
        return None


# Main process to find data, train model, and predict

# Re-fetch instruments data to ensure it's up-to-date and contains options
try:
    print("Attempting to re-fetch instruments data...")
    instruments = kite.instruments("NSE")
    df_instruments = pd.DataFrame(instruments)
    print("✅ Successfully re-fetched instruments data.")

    # Find a valid NIFTY option from df_instruments
    if 'df_instruments' in locals() and not df_instruments.empty:
        # Filter for NIFTY options with expiry dates in the future
        nifty_options = df_instruments[(df_instruments['name'] == 'NIFTY') &
                                      (df_instruments['instrument_type'].isin(['CE', 'PE'])) &
                                      (df_instruments['expiry'].notna()) &
                                      (pd.to_datetime(df_instruments['expiry']) > datetime.now())].copy()


        if not nifty_options.empty:
            # Sort by expiry date to find the nearest upcoming one
            nifty_options['expiry'] = pd.to_datetime(nifty_options['expiry'])
            nifty_options = nifty_options.sort_values(by='expiry')

            # Get the current NIFTY price to find a relevant strike
            # Assuming nifty_quote is available from a previous step
            if 'nifty_quote' in locals():
                current_nifty_price = nifty_quote
            else:
                # Fetch current NIFTY price if not available
                try:
                    nifty_quote_data = kite.ltp("NSE:NIFTY 50")
                    current_nifty_price = nifty_quote_data["NSE:NIFTY 50"]["last_price"]
                    print(f"Fetched current NIFTY price: {current_nifty_price}")
                except Exception as e:
                    print(f"Error fetching NIFTY LTP: {e}")
                    current_nifty_price = None
                    print("Cannot determine a relevant strike price without current NIFTY price.")


            if current_nifty_price is not None:
                # Find the option closest to the current price with the nearest upcoming expiry
                nearest_expiry = nifty_options.iloc[0]['expiry']
                options_on_nearest_expiry = nifty_options[nifty_options['expiry'] == nearest_expiry].copy()

                if not options_on_nearest_expiry.empty:
                    # Find the strike closest to the current price on this expiry
                    options_on_nearest_expiry['strike_diff'] = abs(options_on_nearest_expiry['strike'] - current_nifty_price)
                    closest_option = options_on_nearest_expiry.sort_values(by='strike_diff').iloc[0]

                    valid_expiry = closest_option['expiry'].strftime('%Y-%m-%d')
                    valid_strike = str(int(closest_option['strike']))
                    valid_option_type = closest_option['instrument_type']

                    print(f"\nFound a potentially valid NIFTY option:")
                    print(f"Expiry: {valid_expiry}")
                    print(f"Strike: {valid_strike}")
                    print(f"Type: {valid_option_type}")

                    # Now, attempt to fetch data with the valid parameters
                    print(f"\nAttempting to fetch historical data for {valid_option_type} {valid_strike} {valid_expiry}...")
                    # Pass df_instruments to the fetch_option_data function
                    data = fetch_option_data(kite, df_instruments, "NIFTY", valid_expiry, valid_strike, valid_option_type)

                    # Prepare data and train model if data is fetched
                    if data is not None and not data.empty:
                        df = data[['close']].copy()
                        scaler = MinMaxScaler()
                        scaled_data = scaler.fit_transform(df)

                        time_steps = 50
                        X, y = create_sequences(scaled_data, time_steps)

                        if X.shape[0] > 0:
                            X = X.reshape(X.shape[0], X.shape[1], 1)
                            y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

                            # Define and compile the quantile regression model
                            model_quantile = Sequential()
                            model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                            model_quantile.add(LSTM(units=50))
                            model_quantile.add(Dense(3)) # Output 3 quantiles

                            quantiles = [0.25, 0.50, 0.75]
                            model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                            # Train model
                            print("Training Quantile Regression LSTM model...")
                            model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
                            print("✅ Model training complete.")

                            # Implement prediction and interpretation logic
                            print("\nImplementing prediction and interpretation...")
                            if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
                                last_sequence = scaled_data[-time_steps:]
                                last_sequence = last_sequence.reshape(1, time_steps, 1)

                                predicted_scaled_quantiles = model_quantile.predict(last_sequence)
                                predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

                                # Interpret predicted quantiles
                                q25 = predicted_quantiles[0][0]
                                q50 = predicted_quantiles[0][1]
                                q75 = predicted_quantiles[0][2]

                                # Define high confidence targets
                                target1 = q75 # 75th percentile as Target 1
                                target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

                                # Calculate stop loss
                                entry_price = float(df.iloc[-1]['close'])
                                # Stop loss below the 25th percentile, using distance between 50th and 25th
                                stoploss = q25 - (q50 - q25)


                                print(f"📈 Entry Price: ₹{entry_price:.2f}")
                                print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
                                print(f"🛑 Stoploss: ₹{stoploss:.2f}")
                                print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
                                print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
                            else:
                                print("Cannot make predictions: Not enough data to create the last sequence.")

                        else:
                             print("Not enough data points to create sequences for LSTM training.")

                    else:
                        print("Cannot train the model because historical data fetching with potentially valid parameters failed or returned empty data.")

                else:
                     print("Could not find a suitable NIFTY option on the nearest upcoming expiry.")
            else:
                 print("Current NIFTY price is not available to determine a relevant strike.")
        else:
            print("No NIFTY options with future expiries found in the instruments data after re-fetching.")
    else:
        print("Instruments data (df_instruments) is not available or empty after re-fetching.")

except Exception as e:
    print(f"An error occurred during the process of finding data, training, and predicting: {e}")

Attempting to re-fetch instruments data...
✅ Successfully re-fetched instruments data.
No NIFTY options with future expiries found in the instruments data after re-fetching.


**Reasoning**:
The previous attempt to find a NIFTY option failed because no options with *future* expiries were found in the instruments data based on the current date (2025-08-07). It's possible that the available options have expiries in the past relative to the current date provided. To complete the subtask, I need to find *any* NIFTY option to attempt fetching historical data and proceeding with the model training and prediction. I will modify the filtering logic to find any NIFTY option, regardless of whether its expiry is in the future or past, to check if historical data can be fetched. If data is fetched, I will proceed with training and prediction. If not, the task will fail due to data unavailability.



In [25]:
# Main process to find data, train model, and predict - Retry with less strict expiry filter

try:
    print("\nAttempting to find any NIFTY option (including past expiries) to fetch data...")

    # Find any NIFTY option from df_instruments (including past expiries)
    if 'df_instruments' in locals() and not df_instruments.empty:
        # Filter for NIFTY options with expiry dates (any expiry)
        nifty_options = df_instruments[(df_instruments['name'] == 'NIFTY') &
                                      (df_instruments['instrument_type'].isin(['CE', 'PE'])) &
                                      (df_instruments['expiry'].notna())].copy()


        if not nifty_options.empty:
            # Sort by expiry date (ascending) to pick the earliest available option
            nifty_options['expiry'] = pd.to_datetime(nifty_options['expiry'])
            nifty_options = nifty_options.sort_values(by='expiry')

            # Get the current NIFTY price to find a relevant strike (if available)
            if 'nifty_quote' in locals():
                current_nifty_price = nifty_quote
            else:
                # Fetch current NIFTY price if not available
                try:
                    nifty_quote_data = kite.ltp("NSE:NIFTY 50")
                    current_nifty_price = nifty_quote_data["NSE:NIFTY 50"]["last_price"]
                    print(f"Fetched current NIFTY price: {current_nifty_price}")
                except Exception as e:
                    print(f"Error fetching NIFTY LTP: {e}")
                    current_nifty_price = None
                    print("Cannot determine a relevant strike price without current NIFTY price.")


            if current_nifty_price is not None:
                 # Find the option closest to the current price among all available expiries
                nifty_options['strike_diff'] = abs(nifty_options['strike'] - current_nifty_price)
                closest_option = nifty_options.sort_values(by='strike_diff').iloc[0]

                valid_expiry = closest_option['expiry'].strftime('%Y-%m-%d')
                valid_strike = str(int(closest_option['strike']))
                valid_option_type = closest_option['instrument_type']

                print(f"\nFound a potentially valid NIFTY option (may have past expiry):")
                print(f"Expiry: {valid_expiry}")
                print(f"Strike: {valid_strike}")
                print(f"Type: {valid_option_type}")

                # Now, attempt to fetch data with the valid parameters
                print(f"\nAttempting to fetch historical data for {valid_option_type} {valid_strike} {valid_expiry}...")
                # Pass df_instruments to the fetch_option_data function
                # Assuming kite and df_instruments are available globally or in scope
                data = fetch_option_data(kite, df_instruments, "NIFTY", valid_expiry, valid_strike, valid_option_type)

                # Prepare data and train model if data is fetched
                if data is not None and not data.empty:
                    df = data[['close']].copy()
                    scaler = MinMaxScaler()
                    scaled_data = scaler.fit_transform(df)

                    time_steps = 50
                    X, y = create_sequences(scaled_data, time_steps)

                    if X.shape[0] > 0:
                        X = X.reshape(X.shape[0], X.shape[1], 1)
                        y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

                        # Define and compile the quantile regression model
                        # Assuming model_quantile is not defined globally
                        model_quantile = Sequential()
                        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                        model_quantile.add(LSTM(units=50))
                        model_quantile.add(Dense(3)) # Output 3 quantiles

                        quantiles = [0.25, 0.50, 0.75]
                        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                        # Train model
                        print("Training Quantile Regression LSTM model...")
                        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
                        print("✅ Model training complete.")

                        # Implement prediction and interpretation logic
                        print("\nImplementing prediction and interpretation...")
                        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
                            last_sequence = scaled_data[-time_steps:]
                            last_sequence = last_sequence.reshape(1, time_steps, 1)

                            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
                            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

                            # Interpret predicted quantiles
                            q25 = predicted_quantiles[0][0]
                            q50 = predicted_quantiles[0][1]
                            q75 = predicted_quantiles[0][2]

                            # Define high confidence targets
                            target1 = q75 # 75th percentile as Target 1
                            target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

                            # Calculate stop loss
                            entry_price = float(df.iloc[-1]['close'])
                            # Stop loss below the 25th percentile, using distance between 50th and 25th
                            stoploss = q25 - (q50 - q25)


                            print(f"📈 Entry Price: ₹{entry_price:.2f}")
                            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
                            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
                            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
                            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
                        else:
                            print("Cannot make predictions: Not enough data to create the last sequence.")

                    else:
                        print("Not enough data points to create sequences for LSTM training.")

                else:
                    print("Cannot train the model because historical data fetching with the found option failed or returned empty data.")

            else:
                 print("Current NIFTY price is not available to determine a relevant strike.")
        else:
            print("No NIFTY options found in the instruments data.")
    else:
        print("Instruments data (df_instruments) is not available or empty.")

except Exception as e:
    print(f"An error occurred during the process of finding data, training, and predicting: {e}")



Attempting to find any NIFTY option (including past expiries) to fetch data...
No NIFTY options found in the instruments data.


## Calculate stop loss and targets

### Subtask:
Calculate stop loss and targets


## Add user input cells

### Subtask:
Generate code cells that prompt the user for the required inputs (expiry date, strike, option type).


**Reasoning**:
The previous attempts failed because the necessary data (NIFTY options in `df_instruments`) was not available. The current subtask is to prompt the user for input. This does not depend on the availability of data and can be done independently. I will create a code cell to prompt the user for the expiry date.



In [26]:
user_expiry_date_str = input("Enter the option expiry date in 'YYYY-MM-DD' format: ")
print(f"You entered expiry date: {user_expiry_date_str}")

Enter the option expiry date in 'YYYY-MM-DD' format: 2024-08-07
You entered expiry date: 2024-08-07


**Reasoning**:
I have prompted for the expiry date. Now I need to prompt the user for the strike price as per the instructions.



In [27]:
user_strike_price_str = input("Enter the option strike price: ")
print(f"You entered strike price: {user_strike_price_str}")

Enter the option strike price: 24250
You entered strike price: 24250


**Reasoning**:
I have prompted for the expiry date and strike price. Now I need to prompt the user for the option type as per the instructions.



In [28]:
user_option_type_str = input("Enter the option type ('CE' for Call, 'PE' for Put): ")
print(f"You entered option type: {user_option_type_str}")

Enter the option type ('CE' for Call, 'PE' for Put): CE
You entered option type: CE


## Integrate user input with the workflow

### Subtask:
Integrate the user input cells with the workflow by using the captured inputs (expiry date, strike price, and option type) in the `fetch_option_data` function to retrieve the relevant option data.


**Reasoning**:
Modify the code cell that calls `fetch_option_data` to use the user input variables and add checks for successful data fetching before proceeding with data preparation, training, and prediction.



In [29]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the quantile loss function (assuming this is available)
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences (assuming this is available)
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data (assuming this is available)
# Updated to explicitly pass df_instruments
def fetch_option_data(kite, df_instruments, symbol, expiry_date_str, strike_price_str, option_type_str):
    df = df_instruments.copy() # Use a copy to avoid modifying the original DataFrame
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None


    df = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df = df[df["instrument_type"] == option_type]

    if df.empty:
        print(f"❌ Option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")
        return None

    token = df.iloc[0]["instrument_token"]

    # fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()

    try:
        hist = kite.historical_data(token, from_date, to_date, "30minute")
        df_hist = pd.DataFrame(hist)
        if df_hist.empty:
             print(f"❌ No historical data found for token {token}.")
             return None
        return df_hist
    except Exception as e:
        print(f"Error fetching historical data for token {token}: {e}")
        return None


# Assuming user_expiry_date_str, user_strike_price_str, and user_option_type_str are available from previous cells
# Assuming kite and df_instruments are available globally or in scope

print(f"Attempting to fetch data for NIFTY {user_option_type_str} {user_strike_price_str} expiring on {user_expiry_date_str}...")
data = fetch_option_data(kite, df_instruments, "NIFTY", user_expiry_date_str, user_strike_price_str, user_option_type_str)

# Prepare data and train model if data is fetched
if data is not None and not data.empty:
    df = data[['close']].copy()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    time_steps = 50
    X, y = create_sequences(scaled_data, time_steps)

    if X.shape[0] > 0:
        X = X.reshape(X.shape[0], X.shape[1], 1)
        y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

        # Define and compile the quantile regression model
        model_quantile = Sequential()
        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model_quantile.add(LSTM(units=50))
        model_quantile.add(Dense(3)) # Output 3 quantiles

        quantiles = [0.25, 0.50, 0.75]
        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model...")
        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
        print("✅ Model training complete.")

        # Implement prediction and interpretation logic
        print("\nImplementing prediction and interpretation...")
        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
            last_sequence = scaled_data[-time_steps:]
            last_sequence = last_sequence.reshape(1, time_steps, 1)

            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

            # Interpret predicted quantiles
            q25 = predicted_quantiles[0][0]
            q50 = predicted_quantiles[0][1]
            q75 = predicted_quantiles[0][2]

            # Define high confidence targets
            target1 = q75 # 75th percentile as Target 1
            target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

            # Calculate stop loss
            entry_price = float(df.iloc[-1]['close'])
            # Stop loss below the 25th percentile, using distance between 50th and 25th
            stoploss = q25 - (q50 - q25)


            print(f"📈 Entry Price: ₹{entry_price:.2f}")
            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
        else:
            print("Cannot make predictions: Not enough data to create the last sequence.")

    else:
         print("Not enough data points to create sequences for LSTM training.")

else:
    print("Cannot train the model because data fetching with user parameters failed or returned empty data.")


Attempting to fetch data for NIFTY CE 24250 expiring on 2024-08-07...
❌ Option not found for NIFTY CE 24250.0 expiring on 2024-08-07.
Cannot train the model because data fetching with user parameters failed or returned empty data.


**Reasoning**:
The data fetching failed because the specific option requested by the user was not found in the instruments data. To address this, I will modify the code to first fetch the instruments data again within this block to ensure it is current, then attempt to fetch the user-specified option data. If the specific option is still not found, I will try to find the closest available option based on the user's input expiry, strike, and type, fetch data for that, and proceed with the rest of the workflow.



In [38]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the quantile loss function (assuming this is available)
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences (assuming this is available)
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data (assuming this is available)
# Modified to accept df_instruments and handle not found cases
def fetch_option_data(kite, df_instruments, symbol, expiry_date_str, strike_price_str, option_type_str):
    df = df_instruments.copy() # Use a copy to avoid modifying the original DataFrame
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None


    df_filtered = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df_filtered = df_filtered[df_filtered["instrument_type"] == option_type]

    token = None

    if not df_filtered.empty:
        token = df_filtered.iloc[0]["instrument_token"]
        print(f"Found exact option token: {token}")
    else:
        print(f"❌ Option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")

        # Attempt to find the closest option if the exact one is not found
        print("Attempting to find the closest available option...")
        closest_option = None
        min_diff = float('inf')

        # Filter for NIFTY options with the requested type and expiry
        options_on_expiry_type = df[(df["expiry"] == expiry_date) & (df["instrument_type"] == option_type)].copy()

        if not options_on_expiry_type.empty:
            # Find the strike closest to the requested strike
            options_on_expiry_type['strike_diff'] = abs(options_on_expiry_type['strike'] - strike_price)
            closest_option = options_on_expiry_type.sort_values(by='strike_diff').iloc[0]
            min_diff = closest_option['strike_diff']

            print(f"Found closest option: {closest_option['tradingsymbol']} with strike {closest_option['strike']}")

        if closest_option is not None:
            token = closest_option["instrument_token"]
            print(f"Fetching data for closest option token: {token}")
        else:
            print("Could not find any options with the requested expiry and type.")
            return None # Return None if no closest option is found

    # fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()

    try:
        hist = kite.historical_data(token, from_date, to_date, "30minute")
        df_hist = pd.DataFrame(hist)
        if df_hist.empty:
             print(f"❌ No historical data found for token {token}.")
             return None
        return df_hist
    except Exception as e:
        print(f"Error fetching historical data for token {token}: {e}")
        return None


# Main process to find data, train model, and predict

# Re-fetch instruments data to ensure it's up-to-date and contains options
try:
    print("Attempting to re-fetch instruments data...")
    instruments = kite.instruments("NSE")
    df_instruments = pd.DataFrame(instruments)
    print("✅ Successfully re-fetched instruments data.")

    if 'df_instruments' in locals() and not df_instruments.empty:

        # Assuming user_expiry_date_str, user_strike_price_str, and user_option_type_str are available from previous cells
        print(f"\nAttempting to fetch data for user-specified option: NIFTY {user_option_type_str} {user_strike_price_str} expiring on {user_expiry_date_str}...")
        # Pass df_instruments to the fetch_option_data function
        data = fetch_option_data(kite, df_instruments, "NIFTY", user_expiry_date_str, user_strike_price_str, user_option_type_str)

        # Prepare data and train model if data is fetched
        if data is not None and not data.empty:
            df = data[['close']].copy()
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(df)

            time_steps = 50
            X, y = create_sequences(scaled_data, time_steps)

            if X.shape[0] > 0:
                X = X.reshape(X.shape[0], X.shape[1], 1)
                y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

                # Define and compile the quantile regression model
                model_quantile = Sequential()
                model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                model_quantile.add(LSTM(units=50))
                model_quantile.add(Dense(3)) # Output 3 quantiles

                quantiles = [0.25, 0.50, 0.75]
                model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                # Train model
                print("Training Quantile Regression LSTM model...")
                model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
                print("✅ Model training complete.")

                # Implement prediction and interpretation logic
                print("\nImplementing prediction and interpretation...")
                if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
                    last_sequence = scaled_data[-time_steps:]
                    last_sequence = last_sequence.reshape(1, time_steps, 1)

                    predicted_scaled_quantiles = model_quantile.predict(last_sequence)
                    predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

                    # Interpret predicted quantiles
                    q25 = predicted_quantiles[0][0]
                    q50 = predicted_quantiles[0][1]
                    q75 = predicted_quantiles[0][2]

                    # Define high confidence targets
                    target1 = q75 # 75th percentile as Target 1
                    target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

                    # Calculate stop loss
                    entry_price = float(df.iloc[-1]['close'])
                    # Stop loss below the 25th percentile, using distance between 50th and 25th
                    stoploss = q25 - (q50 - q25)


                    print(f"📈 Entry Price: ₹{entry_price:.2f}")
                    print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
                    print(f"🛑 Stoploss: ₹{stoploss:.2f}")
                    print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
                    print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
                else:
                    print("Cannot make predictions: Not enough data to create the last sequence.")

            else:
                 print("Not enough data points to create sequences for LSTM training.")

        else:
            print("Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.")

    else:
        print("Instruments data (df_instruments) is not available or empty after re-fetching.")

except Exception as e:
    print(f"An error occurred during the process: {e}")

Attempting to re-fetch instruments data...
✅ Successfully re-fetched instruments data.

Attempting to fetch data for user-specified option: NIFTY CE 24250 expiring on 2025-08-07...
❌ Option not found for NIFTY CE 24250.0 expiring on 2025-08-07.
Attempting to find the closest available option...
Could not find any options with the requested expiry and type.
Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.


**Reasoning**:
The previous attempt to fetch data based on user input failed because neither the exact option nor a close alternative with the specified expiry and type could be found in the instruments data. This indicates the instruments data fetched does not contain options matching the user's input criteria. To address this, I will modify the `fetch_option_data` function to be more robust by first re-fetching the instruments data *within* the function itself to ensure it's using the latest data. Then, if the exact option is not found, it will search for the closest option *across all available expiries* with the requested strike and type, prioritizing options near the current date if possible, before giving up. This makes the data fetching step more resilient to variations in available instruments.



In [39]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the quantile loss function (assuming this is available)
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences (assuming this is available)
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data
# Modified to re-fetch instruments and find closest across expiries if exact not found
def fetch_option_data(kite, symbol, expiry_date_str, strike_price_str, option_type_str):
    # Re-fetch instruments data within the function to ensure it's current
    try:
        print("Re-fetching instruments data within fetch_option_data...")
        instruments = kite.instruments("NSE")
        df_instruments_local = pd.DataFrame(instruments)
        print("✅ Successfully re-fetched instruments data within fetch_option_data.")
    except Exception as e:
        print(f"Error fetching instruments data in fetch_option_data: {e}")
        return None

    if df_instruments_local.empty:
        print("❌ Instruments data is empty after re-fetching.")
        return None

    df = df_instruments_local.copy() # Use a copy
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None

    # First, try to find the exact option
    df_exact = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df_exact = df_exact[df_exact["instrument_type"] == option_type]

    token = None
    fetched_option_details = None

    if not df_exact.empty:
        token = df_exact.iloc[0]["instrument_token"]
        fetched_option_details = df_exact.iloc[0]
        print(f"Found exact option token: {token}")
    else:
        print(f"❌ Exact option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")

        # If exact option not found, attempt to find the closest option across ALL expiries
        print("Attempting to find the closest available option across all expiries...")
        closest_option = None
        min_diff = float('inf')

        # Filter for NIFTY options with the requested type
        options_of_type = df[df["instrument_type"] == option_type].copy()

        if not options_of_type.empty:
            # Find the option with the strike closest to the requested strike
            options_of_type['strike_diff'] = abs(options_of_type['strike'] - strike_price)
            # Prioritize options with future expiries if possible
            future_options = options_of_type[options_of_type['expiry'] >= datetime.now()].copy()

            if not future_options.empty:
                 closest_option = future_options.sort_values(by='strike_diff').iloc[0]
            else:
                # If no future options, find the closest among all options of the type
                closest_option = options_of_type.sort_values(by='strike_diff').iloc[0]

            if closest_option is not None:
                token = closest_option["instrument_token"]
                fetched_option_details = closest_option
                print(f"Found closest option: {closest_option['tradingsymbol']} with strike {closest_option['strike']} and expiry {closest_option['expiry'].strftime('%Y-%m-%d')}. Token: {token}")
            else:
                print("Could not find any NIFTY options with the requested type.")
                return None # Return None if no closest option is found
        else:
            print("Could not find any NIFTY options with the requested type.")
            return None # Return None if no options of the requested type are found


    # If a token was found (either exact or closest), fetch historical data
    if token is not None:
        # fetch 30-min candle data for last 10 days
        from_date = datetime.now() - timedelta(days=10)
        to_date = datetime.now()

        try:
            hist = kite.historical_data(token, from_date, to_date, "30minute")
            df_hist = pd.DataFrame(hist)
            if df_hist.empty:
                 print(f"❌ No historical data found for token {token}.")
                 return None
            # Add option details to the dataframe for later use (e.g., entry price context)
            df_hist['instrument_token'] = fetched_option_details['instrument_token']
            df_hist['tradingsymbol'] = fetched_option_details['tradingsymbol']
            df_hist['strike'] = fetched_option_details['strike']
            df_hist['expiry'] = fetched_option_details['expiry']
            df_hist['instrument_type'] = fetched_option_details['instrument_type']

            return df_hist
        except Exception as e:
            print(f"Error fetching historical data for token {token}: {e}")
            return None
    else:
        return None


# --- Workflow Starts Here ---

# 1. Get User Input
user_expiry_date_str = input("Enter the option expiry date in 'YYYY-MM-DD' format: ")
print(f"You entered expiry date: {user_expiry_date_str}")

user_strike_price_str = input("Enter the option strike price: ")
print(f"You entered strike price: {user_strike_price_str}")

user_option_type_str = input("Enter the option type ('CE' for Call, 'PE' for Put): ")
print(f"You entered option type: {user_option_type_str}")

# 2. Fetch Option Data (using the function which handles exact/closest logic)
print(f"\nAttempting to fetch data for user-specified option (or closest): NIFTY {user_option_type_str} {user_strike_price_str} expiring on {user_expiry_date_str}...")
# Assuming kite is available globally or in scope
data = fetch_option_data(kite, "NIFTY", user_expiry_date_str, user_strike_price_str, user_option_type_str)

# 3. Prepare Data and Train Model if data is fetched
if data is not None and not data.empty:
    print("\nData fetched successfully. Preparing data and training model...")
    df = data[['close']].copy()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    time_steps = 50
    X, y = create_sequences(scaled_data, time_steps)

    if X.shape[0] > 0:
        X = X.reshape(X.shape[0], X.shape[1], 1)
        y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

        # Define and compile the quantile regression model
        model_quantile = Sequential()
        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model_quantile.add(LSTM(units=50))
        model_quantile.add(Dense(3)) # Output 3 quantiles

        quantiles = [0.25, 0.50, 0.75]
        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model...")
        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
        print("✅ Model training complete.")

        # 4. Implement prediction and interpretation logic
        print("\nImplementing prediction and interpretation...")
        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
            last_sequence = scaled_data[-time_steps:]
            last_sequence = last_sequence.reshape(1, time_steps, 1)

            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

            # Interpret predicted quantiles
            q25 = predicted_quantiles[0][0]
            q50 = predicted_quantiles[0][1]
            q75 = predicted_quantiles[0][2]

            # Define high confidence targets
            target1 = q75 # 75th percentile as Target 1
            target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

            # Calculate stop loss
            entry_price = float(df.iloc[-1]['close'])
            # Stop loss below the 25th percentile, using distance between 50th and 25th
            stoploss = q25 - (q50 - q25)


            print(f"📈 Entry Price: ₹{entry_price:.2f}")
            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
        else:
            print("Cannot make predictions: Not enough data to create the last sequence.")

    else:
         print("Not enough data points to create sequences for LSTM training.")

else:
    print("Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.")

KeyboardInterrupt: Interrupted by user

## Refine and test

### Subtask:
Refine the code by incorporating the user input cells and the updated data fetching logic into a single, cohesive workflow. Test the entire process with different user inputs, focusing on cases where the exact option is found and where it is not, to ensure the closest option logic and error handling work correctly.


**Reasoning**:
Consolidate the code from user input, data fetching, preparation, training, and prediction into a single workflow. Test with a valid input first.



In [32]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the quantile loss function
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data
def fetch_option_data(kite, symbol, expiry_date_str, strike_price_str, option_type_str):
    # Re-fetch instruments data within the function to ensure it's current
    try:
        print("Re-fetching instruments data within fetch_option_data...")
        instruments = kite.instruments("NSE")
        df_instruments_local = pd.DataFrame(instruments)
        print("✅ Successfully re-fetched instruments data within fetch_option_data.")
    except Exception as e:
        print(f"Error fetching instruments data in fetch_option_data: {e}")
        return None

    if df_instruments_local.empty:
        print("❌ Instruments data is empty after re-fetching.")
        return None

    df = df_instruments_local.copy() # Use a copy
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None

    # First, try to find the exact option
    df_exact = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df_exact = df_exact[df_exact["instrument_type"] == option_type]

    token = None
    fetched_option_details = None

    if not df_exact.empty:
        token = df_exact.iloc[0]["instrument_token"]
        fetched_option_details = df_exact.iloc[0]
        print(f"Found exact option token: {token}")
    else:
        print(f"❌ Exact option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")

        # If exact option not found, attempt to find the closest option across ALL expiries
        print("Attempting to find the closest available option across all expiries...")
        closest_option = None
        min_diff = float('inf')

        # Filter for NIFTY options with the requested type
        options_of_type = df[df["instrument_type"] == option_type].copy()

        if not options_of_type.empty:
            # Find the option with the strike closest to the requested strike
            options_of_type['strike_diff'] = abs(options_of_type['strike'] - strike_price)
            # Prioritize options with future expiries if possible
            future_options = options_of_type[options_of_type['expiry'] >= datetime.now()].copy()

            if not future_options.empty:
                 closest_option = future_options.sort_values(by='strike_diff').iloc[0]
            else:
                # If no future options, find the closest among all options of the type
                closest_option = options_of_type.sort_values(by='strike_diff').iloc[0]

            if closest_option is not None:
                token = closest_option["instrument_token"]
                fetched_option_details = closest_option
                print(f"Found closest option: {closest_option['tradingsymbol']} with strike {closest_option['strike']} and expiry {closest_option['expiry'].strftime('%Y-%m-%d')}. Token: {token}")
            else:
                print("Could not find any NIFTY options with the requested type.")
                return None # Return None if no closest option is found
        else:
            print("Could not find any NIFTY options with the requested type.")
            return None # Return None if no options of the requested type are found


    # If a token was found (either exact or closest), fetch historical data
    if token is not None:
        # fetch 30-min candle data for last 10 days
        from_date = datetime.now() - timedelta(days=10)
        to_date = datetime.now()

        try:
            hist = kite.historical_data(token, from_date, to_date, "30minute")
            df_hist = pd.DataFrame(hist)
            if df_hist.empty:
                 print(f"❌ No historical data found for token {token}.")
                 return None
            # Add option details to the dataframe for later use (e.g., entry price context)
            df_hist['instrument_token'] = fetched_option_details['instrument_token']
            df_hist['tradingsymbol'] = fetched_option_details['tradingsymbol']
            df_hist['strike'] = fetched_option_details['strike']
            df_hist['expiry'] = fetched_option_details['expiry']
            df_hist['instrument_type'] = fetched_option_details['instrument_type']

            return df_hist
        except Exception as e:
            print(f"Error fetching historical data for token {token}: {e}")
            return None
    else:
        return None


# --- Workflow Starts Here ---

# 1. Get User Input
user_expiry_date_str = input("Enter the option expiry date in 'YYYY-MM-DD' format: ")
print(f"You entered expiry date: {user_expiry_date_str}")

user_strike_price_str = input("Enter the option strike price: ")
print(f"You entered strike price: {user_strike_price_str}")

user_option_type_str = input("Enter the option type ('CE' for Call, 'PE' for Put): ")
print(f"You entered option type: {user_option_type_str}")

# 2. Fetch Option Data (using the function which handles exact/closest logic)
print(f"\nAttempting to fetch data for user-specified option (or closest): NIFTY {user_option_type_str} {user_strike_price_str} expiring on {user_expiry_date_str}...")
# Assuming kite is available globally or in scope
data = fetch_option_data(kite, "NIFTY", user_expiry_date_str, user_strike_price_str, user_option_type_str)

# 3. Prepare Data and Train Model if data is fetched
if data is not None and not data.empty:
    print("\nData fetched successfully. Preparing data and training model...")
    df = data[['close']].copy()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    time_steps = 50
    X, y = create_sequences(scaled_data, time_steps)

    if X.shape[0] > 0:
        X = X.reshape(X.shape[0], X.shape[1], 1)
        y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

        # Define and compile the quantile regression model
        model_quantile = Sequential()
        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model_quantile.add(LSTM(units=50))
        model_quantile.add(Dense(3)) # Output 3 quantiles

        quantiles = [0.25, 0.50, 0.75]
        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model...")
        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
        print("✅ Model training complete.")

        # 4. Implement prediction and interpretation logic
        print("\nImplementing prediction and interpretation...")
        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
            last_sequence = scaled_data[-time_steps:]
            last_sequence = last_sequence.reshape(1, time_steps, 1)

            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

            # Interpret predicted quantiles
            q25 = predicted_quantiles[0][0]
            q50 = predicted_quantiles[0][1]
            q75 = predicted_quantiles[0][2]

            # Define high confidence targets
            target1 = q75 # 75th percentile as Target 1
            target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

            # Calculate stop loss
            entry_price = float(df.iloc[-1]['close'])
            # Stop loss below the 25th percentile, using distance between 50th and 25th
            stoploss = q25 - (q50 - q25)

            print(f"📈 Entry Price: ₹{entry_price:.2f}")
            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
        else:
            print("Cannot make predictions: Not enough data to create the last sequence.")

    else:
         print("Not enough data points to create sequences for LSTM training.")

else:
    print("Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.")


Enter the option expiry date in 'YYYY-MM-DD' format: 2025-08-07
You entered expiry date: 2025-08-07
Enter the option strike price: 24250
You entered strike price: 24250
Enter the option type ('CE' for Call, 'PE' for Put): CE
You entered option type: CE

Attempting to fetch data for user-specified option (or closest): NIFTY CE 24250 expiring on 2025-08-07...
Re-fetching instruments data within fetch_option_data...
✅ Successfully re-fetched instruments data within fetch_option_data.
❌ Exact option not found for NIFTY CE 24250.0 expiring on 2025-08-07.
Attempting to find the closest available option across all expiries...
Could not find any NIFTY options with the requested type.
Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.


**Reasoning**:
Test the consolidated workflow with user inputs that are likely to find a closest option, but not an exact match, to verify the closest option logic and ensure the process continues to training and prediction.



In [33]:
# --- Test Case 2: Input that should find a closest option ---
print("\n--- Test Case 2: Input that should find a closest option ---")

# Choose parameters that might not be an exact match but are likely close to available options
# Example: A strike slightly off from a common strike, or a past expiry but a strike that exists.
# Let's try a strike slightly off from a round number, and a recent past expiry.
user_expiry_date_str_test2 = "2025-07-31" # A recent past expiry
user_strike_price_str_test2 = "24370" # A strike likely close to an available one
user_option_type_str_test2 = "CE"

print(f"Attempting to fetch data for user-specified option (or closest): NIFTY {user_option_type_str_test2} {user_strike_price_str_test2} expiring on {user_expiry_date_str_test2}...")
# Assuming kite is available globally or in scope
data_test2 = fetch_option_data(kite, "NIFTY", user_expiry_date_str_test2, user_strike_price_str_test2, user_option_type_str_test2)

# Prepare data and train model if data is fetched for Test Case 2
if data_test2 is not None and not data_test2.empty:
    print("\nData fetched successfully for Test Case 2. Preparing data and training model...")
    df_test2 = data_test2[['close']].copy()
    scaler_test2 = MinMaxScaler()
    scaled_data_test2 = scaler_test2.fit_transform(df_test2)

    time_steps = 50
    X_test2, y_test2 = create_sequences(scaled_data_test2, time_steps)

    if X_test2.shape[0] > 0:
        X_test2 = X_test2.reshape(X_test2.shape[0], X_test2.shape[1], 1)
        y_quantile_test2 = np.repeat(y_test2, 3, axis=1) # Prepare y for quantile loss

        # Define and compile the quantile regression model (re-using the definition)
        model_quantile_test2 = Sequential()
        model_quantile_test2.add(LSTM(units=50, return_sequences=True, input_shape=(X_test2.shape[1], 1)))
        model_quantile_test2.add(LSTM(units=50))
        model_quantile_test2.add(Dense(3)) # Output 3 quantiles

        quantiles = [0.25, 0.50, 0.75]
        model_quantile_test2.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model for Test Case 2...")
        model_quantile_test2.fit(X_test2, y_quantile_test2, epochs=10, batch_size=32)
        print("✅ Model training complete for Test Case 2.")

        # Implement prediction and interpretation logic for Test Case 2
        print("\nImplementing prediction and interpretation for Test Case 2...")
        if 'scaled_data_test2' in locals() and len(scaled_data_test2) >= time_steps:
            last_sequence_test2 = scaled_data_test2[-time_steps:]
            last_sequence_test2 = last_sequence_test2.reshape(1, time_steps, 1)

            predicted_scaled_quantiles_test2 = model_quantile_test2.predict(last_sequence_test2)
            predicted_quantiles_test2 = scaler_test2.inverse_transform(predicted_scaled_quantiles_test2)

            # Interpret predicted quantiles
            q25_test2 = predicted_quantiles_test2[0][0]
            q50_test2 = predicted_quantiles_test2[0][1]
            q75_test2 = predicted_quantiles_test2[0][2]

            # Define high confidence targets
            target1_test2 = q75_test2 # 75th percentile as Target 1
            target2_test2 = q50_test2 + (q75_test2 - q50_test2) / 2 # Midpoint between 50th and 75th as Target 2

            # Calculate stop loss
            entry_price_test2 = float(df_test2.iloc[-1]['close'])
            # Stop loss below the 25th percentile, using distance between 50th and 25th
            stoploss_test2 = q25_test2 - (q50_test2 - q25_test2)


            print(f"📈 Entry Price (Test Case 2): ₹{entry_price_test2:.2f}")
            print(f"🔮 Predicted Quantiles (Test Case 2 - 25th, 50th, 75th): ₹{q25_test2:.2f}, ₹{q50_test2:.2f}, ₹{q75_test2:.2f}")
            print(f"🛑 Stoploss (Test Case 2): ₹{stoploss_test2:.2f}")
            print(f"🎯 Target 1 (Test Case 2 - High Confidence - 75th percentile): ₹{target1_test2:.2f}")
            print(f"🎯 Target 2 (Test Case 2 - High Confidence - Derived): ₹{target2_test2:.2f}")
        else:
            print("Cannot make predictions for Test Case 2: Not enough data to create the last sequence.")

    else:
         print("Not enough data points to create sequences for LSTM training for Test Case 2.")

else:
    print("Cannot train the model for Test Case 2 because data fetching with user parameters (or closest option) failed or returned empty data.")


--- Test Case 2: Input that should find a closest option ---
Attempting to fetch data for user-specified option (or closest): NIFTY CE 24370 expiring on 2025-07-31...
Re-fetching instruments data within fetch_option_data...
✅ Successfully re-fetched instruments data within fetch_option_data.
❌ Exact option not found for NIFTY CE 24370.0 expiring on 2025-07-31.
Attempting to find the closest available option across all expiries...
Could not find any NIFTY options with the requested type.
Cannot train the model for Test Case 2 because data fetching with user parameters (or closest option) failed or returned empty data.


**Reasoning**:
Test the consolidated workflow with user inputs for which no relevant option (exact or closest) is likely to be found, to verify that the appropriate error messages are displayed and the process terminates gracefully. This will test the failure path more thoroughly.



In [34]:
# --- Test Case 3: Input that should not find any relevant option ---
print("\n--- Test Case 3: Input that should not find any relevant option ---")

# Choose parameters that are highly unlikely to exist
# Example: A non-standard strike, or an expiry far in the past or future, and a type that might not be available.
user_expiry_date_str_test3 = "2020-01-01" # Expiry far in the past
user_strike_price_str_test3 = "5000" # Very low strike
user_option_type_str_test3 = "PE" # PE type

print(f"Attempting to fetch data for user-specified option (or closest): NIFTY {user_option_type_str_test3} {user_strike_price_str_test3} expiring on {user_expiry_date_str_test3}...")
# Assuming kite is available globally or in scope
data_test3 = fetch_option_data(kite, "NIFTY", user_expiry_date_str_test3, user_strike_price_str_test3, user_option_type_str_test3)

# Prepare data and train model if data is fetched for Test Case 3
if data_test3 is not None and not data_test3.empty:
    print("\nData fetched successfully for Test Case 3. Preparing data and training model...")
    df_test3 = data_test3[['close']].copy()
    scaler_test3 = MinMaxScaler()
    scaled_data_test3 = scaler_test3.fit_transform(df_test3)

    time_steps = 50
    X_test3, y_test3 = create_sequences(scaled_data_test3, time_steps)

    if X_test3.shape[0] > 0:
        X_test3 = X_test3.reshape(X_test3.shape[0], X_test3.shape[1], 1)
        y_quantile_test3 = np.repeat(y_test3, 3, axis=1) # Prepare y for quantile loss

        # Define and compile the quantile regression model (re-using the definition)
        model_quantile_test3 = Sequential()
        model_quantile_test3.add(LSTM(units=50, return_sequences=True, input_shape=(X_test3.shape[1], 1)))
        model_quantile_test3.add(LSTM(units=50))
        model_quantile_test3.add(Dense(3)) # Output 3 quantiles

        quantiles = [0.25, 0.50, 0.75]
        model_quantile_test3.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model for Test Case 3...")
        model_quantile_test3.fit(X_test3, y_quantile_test3, epochs=10, batch_size=32)
        print("✅ Model training complete for Test Case 3.")

        # Implement prediction and interpretation logic for Test Case 3
        print("\nImplementing prediction and interpretation for Test Case 3...")
        if 'scaled_data_test3' in locals() and len(scaled_data_test3) >= time_steps:
            last_sequence_test3 = scaled_data_test3[-time_steps:]
            last_sequence_test3 = last_sequence_test3.reshape(1, time_steps, 1)

            predicted_scaled_quantiles_test3 = model_quantile_test3.predict(last_sequence_test3)
            predicted_quantiles_test3 = scaler_test3.inverse_transform(predicted_scaled_quantiles_test3)

            # Interpret predicted quantiles
            q25_test3 = predicted_quantiles_test3[0][0]
            q50_test3 = predicted_quantiles_test3[0][1]
            q75_test3 = predicted_quantiles_test3[0][2]

            # Define high confidence targets
            target1_test3 = q75_test3 # 75th percentile as Target 1
            target2_test3 = q50_test3 + (q75_test3 - q50_test3) / 2 # Midpoint between 50th and 75th as Target 2

            # Calculate stop loss
            entry_price_test3 = float(df_test3.iloc[-1]['close'])
            # Stop loss below the 25th percentile, using distance between 50th and 25th
            stoploss_test3 = q25_test3 - (q50_test3 - q25_test3)


            print(f"📈 Entry Price (Test Case 3): ₹{entry_price_test3:.2f}")
            print(f"🔮 Predicted Quantiles (Test Case 3 - 25th, 50th, 75th): ₹{q25_test3:.2f}, ₹{q50_test3:.2f}, ₹{q75_test3:.2f}")
            print(f"🛑 Stoploss (Test Case 3): ₹{stoploss_test3:.2f}")
            print(f"🎯 Target 1 (Test Case 3 - High Confidence - 75th percentile): ₹{target1_test3:.2f}")
            print(f"🎯 Target 2 (Test Case 3 - High Confidence - Derived): ₹{target2_test3:.2f}")
        else:
            print("Cannot make predictions for Test Case 3: Not enough data to create the last sequence.")

    else:
         print("Not enough data points to create sequences for LSTM training for Test Case 3.")

else:
    print("Cannot train the model for Test Case 3 because data fetching with user parameters (or closest option) failed or returned empty data.")


--- Test Case 3: Input that should not find any relevant option ---
Attempting to fetch data for user-specified option (or closest): NIFTY PE 5000 expiring on 2020-01-01...
Re-fetching instruments data within fetch_option_data...
✅ Successfully re-fetched instruments data within fetch_option_data.
❌ Exact option not found for NIFTY PE 5000.0 expiring on 2020-01-01.
Attempting to find the closest available option across all expiries...
Could not find any NIFTY options with the requested type.
Cannot train the model for Test Case 3 because data fetching with user parameters (or closest option) failed or returned empty data.


In [35]:
user_expiry_date_str = input("Enter the option expiry date in 'YYYY-MM-DD' format: ")
print(f"You entered expiry date: {user_expiry_date_str}")

Enter the option expiry date in 'YYYY-MM-DD' format: 2025-08-07
You entered expiry date: 2025-08-07


In [36]:
user_strike_price_str = input("Enter the option strike price: ")
print(f"You entered strike price: {user_strike_price_str}")

Enter the option strike price: 24250
You entered strike price: 24250


In [37]:
user_option_type_str = input("Enter the option type ('CE' for Call, 'PE' for Put): ")
print(f"You entered option type: {user_option_type_str}")

Enter the option type ('CE' for Call, 'PE' for Put): CE
You entered option type: CE


## Integrate user input with the workflow

### Subtask:
Integrate the user input cells with the workflow by using the captured inputs (expiry date, strike price, and option type) in the `fetch_option_data` function to retrieve the relevant option data.

## Refine and test

### Subtask:
Refine the code by incorporating the user input cells and the updated data fetching logic into a single, cohesive workflow. Test the entire process with different user inputs, focusing on cases where the exact option is found and where it is not, to ensure the closest option logic and error handling work correctly.

In [40]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the quantile loss function
def quantile_loss(quantiles):
    def loss(y_true, y_pred):
        error = y_true - y_pred
        losses = []
        for i, q in enumerate(quantiles):
            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
        return tf.add_n(losses)
    return loss

# Function to create sequences
def create_sequences(data, time_steps=50):
    X, y = [], []
    if len(data) < time_steps:
        return np.array(X), np.array(y)

    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Function to fetch option data
def fetch_option_data(kite, symbol, expiry_date_str, strike_price_str, option_type_str):
    # Re-fetch instruments data within the function to ensure it's current
    try:
        print("Re-fetching instruments data within fetch_option_data...")
        instruments = kite.instruments("NSE")
        df_instruments_local = pd.DataFrame(instruments)
        print("✅ Successfully re-fetched instruments data within fetch_option_data.")
    except Exception as e:
        print(f"Error fetching instruments data in fetch_option_data: {e}")
        return None

    if df_instruments_local.empty:
        print("❌ Instruments data is empty after re-fetching.")
        return None

    df = df_instruments_local.copy() # Use a copy
    df = df[df["name"] == "NIFTY"]

    # Convert user inputs
    try:
        expiry_date = pd.to_datetime(expiry_date_str)
        strike_price = float(strike_price_str)
        option_type = option_type_str.upper()
    except ValueError as e:
        print(f"❌ Invalid input format: {e}")
        return None

    # First, try to find the exact option
    df_exact = df[(df["strike"] == strike_price) & (df["expiry"] == expiry_date)]
    df_exact = df_exact[df_exact["instrument_type"] == option_type]

    token = None
    fetched_option_details = None

    if not df_exact.empty:
        token = df_exact.iloc[0]["instrument_token"]
        fetched_option_details = df_exact.iloc[0]
        print(f"Found exact option token: {token}")
    else:
        print(f"❌ Exact option not found for NIFTY {option_type} {strike_price} expiring on {expiry_date_str}.")

        # If exact option not found, attempt to find the closest option across ALL expiries
        print("Attempting to find the closest available option across all expiries...")
        closest_option = None
        min_diff = float('inf')

        # Filter for NIFTY options with the requested type
        options_of_type = df[df["instrument_type"] == option_type].copy()

        if not options_of_type.empty:
            # Find the option with the strike closest to the requested strike
            options_of_type['strike_diff'] = abs(options_of_type['strike'] - strike_price)
            # Prioritize options with future expiries if possible
            future_options = options_of_type[options_of_type['expiry'] >= datetime.now()].copy()

            if not future_options.empty:
                 closest_option = future_options.sort_values(by='strike_diff').iloc[0]
            else:
                # If no future options, find the closest among all options of the type
                closest_option = options_of_type.sort_values(by='strike_diff').iloc[0]

            if closest_option is not None:
                token = closest_option["instrument_token"]
                fetched_option_details = closest_option
                print(f"Found closest option: {closest_option['tradingsymbol']} with strike {closest_option['strike']} and expiry {closest_option['expiry'].strftime('%Y-%m-%d')}. Token: {token}")
            else:
                print("Could not find any NIFTY options with the requested type.")
                return None # Return None if no closest option is found
        else:
            print("Could not find any NIFTY options with the requested type.")
            return None # Return None if no options of the requested type are found


    # If a token was found (either exact or closest), fetch historical data
    if token is not None:
        # fetch 30-min candle data for last 10 days
        from_date = datetime.now() - timedelta(days=10)
        to_date = datetime.now()

        try:
            hist = kite.historical_data(token, from_date, to_date, "30minute")
            df_hist = pd.DataFrame(hist)
            if df_hist.empty:
                 print(f"❌ No historical data found for token {token}.")
                 return None
            # Add option details to the dataframe for later use (e.g., entry price context)
            df_hist['instrument_token'] = fetched_option_details['instrument_token']
            df_hist['tradingsymbol'] = fetched_option_details['tradingsymbol']
            df_hist['strike'] = fetched_option_details['strike']
            df_hist['expiry'] = fetched_option_details['expiry']
            df_hist['instrument_type'] = fetched_option_details['instrument_type']

            return df_hist
        except Exception as e:
            print(f"Error fetching historical data for token {token}: {e}")
            return None
    else:
        return None


# --- Workflow Starts Here ---

# 1. Get User Input
user_expiry_date_str = input("Enter the option expiry date in 'YYYY-MM-DD' format: ")
print(f"You entered expiry date: {user_expiry_date_str}")

user_strike_price_str = input("Enter the option strike price: ")
print(f"You entered strike price: {user_strike_price_str}")

user_option_type_str = input("Enter the option type ('CE' for Call, 'PE' for Put): ")
print(f"You entered option type: {user_option_type_str}")

# 2. Fetch Option Data (using the function which handles exact/closest logic)
print(f"\nAttempting to fetch data for user-specified option (or closest): NIFTY {user_option_type_str} {user_strike_price_str} expiring on {user_expiry_date_str}...")
# Assuming kite is available globally or in scope
data = fetch_option_data(kite, "NIFTY", user_expiry_date_str, user_strike_price_str, user_option_type_str)

# 3. Prepare Data and Train Model if data is fetched
if data is not None and not data.empty:
    print("\nData fetched successfully. Preparing data and training model...")
    df = data[['close']].copy()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    time_steps = 50
    X, y = create_sequences(scaled_data, time_steps)

    if X.shape[0] > 0:
        X = X.reshape(X.shape[0], X.shape[1], 1)
        y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss

        # Define and compile the quantile regression model
        model_quantile = Sequential()
        model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
        model_quantile.add(LSTM(units=50))
        model_quantile.add(Dense(3)) # Output 3 quantiles

        quantiles = [0.25, 0.50, 0.75]
        model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

        # Train model
        print("Training Quantile Regression LSTM model...")
        model_quantile.fit(X, y_quantile, epochs=10, batch_size=32)
        print("✅ Model training complete.")

        # 4. Implement prediction and interpretation logic
        print("\nImplementing prediction and interpretation...")
        if 'scaled_data' in locals() and len(scaled_data) >= time_steps:
            last_sequence = scaled_data[-time_steps:]
            last_sequence = last_sequence.reshape(1, time_steps, 1)

            predicted_scaled_quantiles = model_quantile.predict(last_sequence)
            predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

            # Interpret predicted quantiles
            q25 = predicted_quantiles[0][0]
            q50 = predicted_quantiles[0][1]
            q75 = predicted_quantiles[0][2]

            # Define high confidence targets
            target1 = q75 # 75th percentile as Target 1
            target2 = q50 + (q75 - q50) / 2 # Midpoint between 50th and 75th as Target 2

            # Calculate stop loss
            entry_price = float(df.iloc[-1]['close'])
            # Stop loss below the 25th percentile, using distance between 50th and 25th
            stoploss = q25 - (q50 - q25)


            print(f"📈 Entry Price: ₹{entry_price:.2f}")
            print(f"🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
            print(f"🛑 Stoploss: ₹{stoploss:.2f}")
            print(f"🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
            print(f"🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
        else:
            print("Cannot make predictions: Not enough data to create the last sequence.")

    else:
         print("Not enough data points to create sequences for LSTM training.")

else:
    print("Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.")

Enter the option expiry date in 'YYYY-MM-DD' format: 2025-08-07
You entered expiry date: 2025-08-07
Enter the option strike price: 24650
You entered strike price: 24650
Enter the option type ('CE' for Call, 'PE' for Put): CE
You entered option type: CE

Attempting to fetch data for user-specified option (or closest): NIFTY CE 24650 expiring on 2025-08-07...
Re-fetching instruments data within fetch_option_data...
✅ Successfully re-fetched instruments data within fetch_option_data.
❌ Exact option not found for NIFTY CE 24650.0 expiring on 2025-08-07.
Attempting to find the closest available option across all expiries...
Could not find any NIFTY options with the requested type.
Cannot train the model because data fetching with user parameters (or closest option) failed or returned empty data.


In [41]:
# Investigate why NIFTY options are not found by filtering the full instruments list
print("Investigating NIFTY options availability in the full instruments list...")

if 'df_instruments' in globals() and df_instruments is not None and not df_instruments.empty:
    # Filter for instruments with 'NIFTY' in the name and instrument_type 'CE' or 'PE'
    all_nifty_options = df_instruments[(df_instruments['name'].str.contains('NIFTY', na=False)) &
                                       (df_instruments['instrument_type'].isin(['CE', 'PE']))].copy()

    if not all_nifty_options.empty:
        print(f"Found {len(all_nifty_options)} instruments related to NIFTY options.")
        print("Displaying the first 10 rows of NIFTY options found:")
        display(all_nifty_options.head(10))
    else:
        print("No instruments found in the full df_instruments list with 'NIFTY' in name and instrument_type 'CE' or 'PE'.")
        print("This confirms that NIFTY options are not being returned by kite.instruments('NSE').")
else:
    print("df_instruments DataFrame is not available or is empty.")

Investigating NIFTY options availability in the full instruments list...
No instruments found in the full df_instruments list with 'NIFTY' in name and instrument_type 'CE' or 'PE'.
This confirms that NIFTY options are not being returned by kite.instruments('NSE').


In [43]:
# Fetch and display the full list of instruments
print("Fetching the full list of instruments...")
try:
    instruments = kite.instruments() # Fetch instruments from all exchanges
    df_full_instruments = pd.DataFrame(instruments)
    print(f"✅ Successfully fetched {len(df_full_instruments)} instruments.")
    display(df_full_instruments.head())
    # Optional: Display info to see column types and non-null counts
    # display(df_full_instruments.info())
except Exception as e:
    print(f"Error fetching full instruments list: {e}")

Fetching the full list of instruments...
✅ Successfully fetched 88028 instruments.


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,211672581,826846,BANKEX25AUGFUT,BANKEX,0.0,2025-08-26,0.0,0.05,30,FUT,BFO-FUT,BFO
1,215681797,842507,BANKEX25SEPFUT,BANKEX,0.0,2025-09-25,0.0,0.05,30,FUT,BFO-FUT,BFO
2,220537093,861473,BANKEX25OCTFUT,BANKEX,0.0,2025-10-30,0.0,0.05,30,FUT,BFO-FUT,BFO
3,215550981,841996,SENSEX25812FUT,SENSEX,0.0,2025-08-12,0.0,0.05,20,FUT,BFO-FUT,BFO
4,211640069,826719,SENSEX25AUGFUT,SENSEX,0.0,2025-08-26,0.0,0.05,20,FUT,BFO-FUT,BFO


from matplotlib import pyplot as plt
_df_0['instrument_token'].plot(kind='hist', bins=20, title='instrument_token')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['lot_size'].plot(kind='hist', bins=20, title='lot_size')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('exchange_token').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['tradingsymbol'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='instrument_token', y='tradingsymbol', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='instrument_token', y='name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['expiry'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='instrument_token', y='expiry', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [45]:
# Save the full instruments DataFrame to a CSV file
csv_filename = "full_kite_instruments.csv"
df_full_instruments.to_csv(csv_filename, index=False)

print(f"✅ Full instruments data saved to {csv_filename}")

✅ Full instruments data saved to full_kite_instruments.csv


# Task
Load the instrument data from "/content/full_kite_instruments.csv", find NIFTY option contracts, allow the user to select a specific option, retrieve its instrument token, fetch historical data using `kite.historical_data()`, train a quantile regression LSTM model on the data to predict entry price, target 1, target 2, and stop loss, and display the results.

## Load instruments data

### Subtask:
Load the `full_kite_instruments.csv` file into a pandas DataFrame.


**Reasoning**:
The subtask is to load the `full_kite_instruments.csv` file into a pandas DataFrame and display its head and info. This requires using pandas to read the CSV file and then using the DataFrame's methods to display the information.



In [46]:
import pandas as pd

csv_filename = "full_kite_instruments.csv"
df_full_instruments = pd.read_csv(csv_filename)

print("First 5 rows of the DataFrame:")
display(df_full_instruments.head())

print("\nDataFrame Information:")
display(df_full_instruments.info())

First 5 rows of the DataFrame:


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,211672581,826846,BANKEX25AUGFUT,BANKEX,0.0,2025-08-26,0.0,0.05,30,FUT,BFO-FUT,BFO
1,215681797,842507,BANKEX25SEPFUT,BANKEX,0.0,2025-09-25,0.0,0.05,30,FUT,BFO-FUT,BFO
2,220537093,861473,BANKEX25OCTFUT,BANKEX,0.0,2025-10-30,0.0,0.05,30,FUT,BFO-FUT,BFO
3,215550981,841996,SENSEX25812FUT,SENSEX,0.0,2025-08-12,0.0,0.05,20,FUT,BFO-FUT,BFO
4,211640069,826719,SENSEX25AUGFUT,SENSEX,0.0,2025-08-26,0.0,0.05,20,FUT,BFO-FUT,BFO



DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88028 entries, 0 to 88027
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instrument_token  88028 non-null  int64  
 1   exchange_token    88028 non-null  int64  
 2   tradingsymbol     88028 non-null  object 
 3   name              80106 non-null  object 
 4   last_price        88028 non-null  float64
 5   expiry            67156 non-null  object 
 6   strike            88028 non-null  float64
 7   tick_size         88028 non-null  float64
 8   lot_size          88028 non-null  int64  
 9   instrument_type   88028 non-null  object 
 10  segment           88028 non-null  object 
 11  exchange          88028 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 8.1+ MB


None

**Reasoning**:
The previous step successfully loaded the CSV data into `df_full_instruments` and displayed its info. The next logical step according to the overall task is to find NIFTY option contracts within this loaded DataFrame. This involves filtering the DataFrame based on the 'name' and 'instrument_type' columns.



In [47]:
# Find NIFTY option contracts (CE and PE)
nifty_options = df_full_instruments[(df_full_instruments['name'] == 'NIFTY') &
                                   (df_full_instruments['instrument_type'].isin(['CE', 'PE']))].copy()

print("Found NIFTY option contracts:")
display(nifty_options.head())
print(f"\nTotal NIFTY option contracts found: {len(nifty_options)}")

Found NIFTY option contracts:


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
50961,10199298,39841,NIFTY2580724550CE,NIFTY,0.0,2025-08-07,24550.0,0.05,75,CE,NFO-OPT,NFO
50962,10199554,39842,NIFTY2580724550PE,NIFTY,0.0,2025-08-07,24550.0,0.05,75,PE,NFO-OPT,NFO
50963,10200322,39845,NIFTY2580724600CE,NIFTY,0.0,2025-08-07,24600.0,0.05,75,CE,NFO-OPT,NFO
50964,10200578,39846,NIFTY2580724600PE,NIFTY,0.0,2025-08-07,24600.0,0.05,75,PE,NFO-OPT,NFO
50965,10196738,39831,NIFTY2580724500CE,NIFTY,0.0,2025-08-07,24500.0,0.05,75,CE,NFO-OPT,NFO



Total NIFTY option contracts found: 1527


**Reasoning**:
I have successfully loaded the instruments data and filtered for NIFTY options. The next step according to the overall task is to allow the user to select a specific option. I will implement this by prompting the user for the expiry date, strike price, and option type, similar to the previous attempts, but now using the loaded `nifty_options` DataFrame to validate inputs or find the closest match.



In [49]:
# Prompt user for input
user_expiry_date_str = input("Enter the option expiry date (YYYY-MM-DD): ")
user_strike_price_str = input("Enter the option strike price: ")
user_option_type_str = input("Enter the option type (CE or PE): ").upper()

print(f"\nYou entered: Expiry={user_expiry_date_str}, Strike={user_strike_price_str}, Type={user_option_type_str}")

# Convert user inputs to appropriate types
try:
    user_expiry_date = pd.to_datetime(user_expiry_date_str)
    user_strike_price = float(user_strike_price_str)
except ValueError as e:
    print(f"❌ Invalid input format: {e}")
    user_expiry_date = None
    user_strike_price = None

selected_option = None
instrument_token = None

# Ensure expiry is datetime in nifty_options for consistent comparison
if 'nifty_options' in locals() and not nifty_options.empty:
    nifty_options['expiry'] = pd.to_datetime(nifty_options['expiry'])

# Find the exact option in the filtered DataFrame
if user_expiry_date is not None and user_strike_price is not None:
    exact_match = nifty_options[(nifty_options['expiry'] == user_expiry_date) &
                                (nifty_options['strike'] == user_strike_price) &
                                (nifty_options['instrument_type'] == user_option_type_str)]

    if not exact_match.empty:
        selected_option = exact_match.iloc[0]
        instrument_token = selected_option['instrument_token']
        print(f"✅ Found exact match: {selected_option['tradingsymbol']}")
    else:
        print("❌ Exact option not found. Attempting to find the closest option.")

        # Find the closest option based on user input if exact match not found
        # Filter for the requested option type first
        options_of_type = nifty_options[nifty_options['instrument_type'] == user_option_type_str].copy()

        if not options_of_type.empty:
            # Filter for the requested expiry date
            options_on_expiry = options_of_type[options_of_type['expiry'] == user_expiry_date].copy()

            if not options_on_expiry.empty:
                # Find the closest strike on the specified expiry
                options_on_expiry['strike_diff'] = abs(options_on_expiry['strike'] - user_strike_price)
                closest_option = options_on_expiry.sort_values(by='strike_diff').iloc[0]
                selected_option = closest_option
                instrument_token = selected_option['instrument_token']
                # Ensure expiry is datetime before formatting
                selected_option['expiry'] = pd.to_datetime(selected_option['expiry'])
                print(f"✅ Found closest option on specified expiry: {selected_option['tradingsymbol']} (Strike: {selected_option['strike']}, Expiry: {selected_option['expiry'].strftime('%Y-%m-%d')})")
            else:
                print("Could not find any options with the specified expiry and type. Attempting to find closest strike across all expiries of the requested type.")
                # Find the closest strike across all expiries of the requested type
                # Ensure expiry is datetime in options_of_type before calculating strike_diff (if needed, though strike_diff is on strike)
                options_of_type['expiry'] = pd.to_datetime(options_of_type['expiry'])
                options_of_type['strike_diff'] = abs(options_of_type['strike'] - user_strike_price)
                closest_option = options_of_type.sort_values(by='strike_diff').iloc[0]
                selected_option = closest_option
                instrument_token = selected_option['instrument_token']
                # Ensure expiry is datetime before formatting
                selected_option['expiry'] = pd.to_datetime(selected_option['expiry'])
                print(f"✅ Found closest option across all expiries: {selected_option['tradingsymbol']} (Strike: {selected_option['strike']}, Expiry: {selected_option['expiry'].strftime('%Y-%m-%d')})")

        else:
            print("Could not find any NIFTY options of the requested type.")

# Check if an instrument token was found before proceeding
if instrument_token is None:
    print("\n❌ Could not find a suitable instrument token based on your input.")
else:
    print(f"\nSelected Instrument Token: {instrument_token}")
    # Store selected option details for later use
    selected_option_details = selected_option.to_dict()
    print("Selected option details stored.")

Enter the option expiry date (YYYY-MM-DD): 2025-08-07
Enter the option strike price: 24700
Enter the option type (CE or PE): CE

You entered: Expiry=2025-08-07, Strike=24700, Type=CE
✅ Found exact match: NIFTY2580724700CE

Selected Instrument Token: 10202626
Selected option details stored.


## Fetch historical data

### Subtask:
Use the retrieved `instrument_token` and the user's desired date range/interval to fetch historical data using `kite.historical_data()`.

In [50]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming instrument_token and selected_option_details are available from previous steps
if 'instrument_token' in locals() and instrument_token is not None:
    print(f"\nAttempting to fetch historical data for instrument token: {instrument_token}...")

    # Fetch 30-min candle data for last 10 days
    from_date = datetime.now() - timedelta(days=10)
    to_date = datetime.now()
    interval = "30minute"

    try:
        hist = kite.historical_data(instrument_token, from_date, to_date, interval)
        data = pd.DataFrame(hist)

        if data is not None and not data.empty:
            print("✅ Historical data fetched successfully.")
            # Add selected option details to the data DataFrame for context in later steps
            data['instrument_token'] = selected_option_details['instrument_token']
            data['tradingsymbol'] = selected_option_details['tradingsymbol']
            data['strike'] = selected_option_details['strike']
            data['expiry'] = selected_option_details['expiry']
            data['instrument_type'] = selected_option_details['instrument_type']


            # --- Step 6: Prepare data ---
            print("\nPreparing data for LSTM model...")
            df = data[['close']].copy()
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(df)

            time_steps = 50 # Define time_steps for sequence creation
            X, y = create_sequences(scaled_data, time_steps)

            if X.shape[0] > 0:
                X = X.reshape(X.shape[0], X.shape[1], 1)
                y_quantile = np.repeat(y, 3, axis=1) # Prepare y for quantile loss (predicting 3 quantiles)

                # --- Step 7: Train LSTM model ---
                print("Training Quantile Regression LSTM model...")

                # Define the quantile loss function (if not already defined)
                def quantile_loss(quantiles):
                    def loss(y_true, y_pred):
                        error = y_true - y_pred
                        losses = []
                        for i, q in enumerate(quantiles):
                            losses.append(tf.reduce_mean(tf.maximum(q * error[:, i], (q - 1) * error[:, i]), axis=-1))
                        return tf.add_n(losses)
                    return loss

                # Define and compile the quantile regression model
                model_quantile = Sequential()
                model_quantile.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
                model_quantile.add(LSTM(units=50))
                model_quantile.add(Dense(3)) # Output 3 quantiles (25th, 50th, 75th)

                quantiles = [0.25, 0.50, 0.75]
                model_quantile.compile(optimizer='adam', loss=quantile_loss(quantiles))

                # Train model
                model_quantile.fit(X, y_quantile, epochs=10, batch_size=32, verbose=0) # Set verbose to 0 to reduce training output
                print("✅ Model training complete.")

                # --- Step 8: Predict targets and stop loss ---
                print("\nPredicting targets and stop loss...")
                if len(scaled_data) >= time_steps:
                    last_sequence = scaled_data[-time_steps:]
                    last_sequence = last_sequence.reshape(1, time_steps, 1)

                    predicted_scaled_quantiles = model_quantile.predict(last_sequence, verbose=0) # Set verbose to 0
                    predicted_quantiles = scaler.inverse_transform(predicted_scaled_quantiles)

                    # Interpret predicted quantiles
                    q25 = predicted_quantiles[0][0]
                    q50 = predicted_quantiles[0][1]
                    q75 = predicted_quantiles[0][2]

                    # Define high confidence targets
                    target1 = q75 # 75th percentile as Target 1
                    target2 = q50 + (q75 - q50) / 2 # Example: Midpoint between 50th and 75th

                    # Calculate stop loss
                    # Entry price is the last closing price from the fetched data
                    entry_price = float(df.iloc[-1]['close'])
                    # Stop loss below the 25th percentile, using distance between 50th and 25th
                    stoploss = q25 - (q50 - q25)


                    # --- Step 9: Display results ---
                    print("\n📈 Trading Recommendation:")
                    print(f"  Instrument: {selected_option_details['tradingsymbol']}")
                    print(f"  Expiry: {selected_option_details['expiry'].strftime('%Y-%m-%d')}")
                    print(f"  Strike: {selected_option_details['strike']:.2f}")
                    print(f"  Type: {selected_option_details['instrument_type']}")
                    print(f"  Entry Price: ₹{entry_price:.2f}")
                    print(f"  🔮 Predicted Quantiles (25th, 50th, 75th): ₹{q25:.2f}, ₹{q50:.2f}, ₹{q75:.2f}")
                    print(f"  🛑 Stoploss: ₹{stoploss:.2f}")
                    print(f"  🎯 Target 1 (High Confidence - 75th percentile): ₹{target1:.2f}")
                    print(f"  🎯 Target 2 (High Confidence - Derived): ₹{target2:.2f}")
                else:
                    print("Cannot make predictions: Not enough data points after creating sequences.")

            else:
                 print("Not enough data points to create sequences for LSTM training.")

        else:
            print("❌ Historical data fetching returned empty data.")

    except Exception as e:
        print(f"Error fetching historical data: {e}")

else:
    print("\n❌ Instrument token not found. Please run the previous step to select an option and get the instrument token.")


Attempting to fetch historical data for instrument token: 10202626...
✅ Historical data fetched successfully.

Preparing data for LSTM model...
Training Quantile Regression LSTM model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model training complete.

Predicting targets and stop loss...

📈 Trading Recommendation:
  Instrument: NIFTY2580724700CE
  Expiry: 2025-08-07
  Strike: 24700.00
  Type: CE
  Entry Price: ₹5.30
  🔮 Predicted Quantiles (25th, 50th, 75th): ₹35.99, ₹39.06, ₹43.53
  🛑 Stoploss: ₹32.92
  🎯 Target 1 (High Confidence - 75th percentile): ₹43.53
  🎯 Target 2 (High Confidence - Derived): ₹41.30


## Finish task

### Subtask:
Summarize the changes made to the notebook and provide instructions on how to use the complete workflow to get NIFTY option price predictions.